In [1]:
import numpy as np
import pandas as pd
import re
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
import scipy.stats as stats
from scipy import math
from scipy import spatial
from functools import reduce 
from itertools import permutations , combinations
from cnsenti import Emotion
import jieba
import jieba.posseg
import collections
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set()

In [2]:
def language_range_coding(hex_strings):
    # 转换16进制语言的unicode列表范围为int
    rangelist = [int("{}{}".format('0x',i),16) for i in hex_strings.split('-')]
    return range(rangelist[0],rangelist[-1])


chinses_unicode_space = re.compile(r'[\u0061-\u007a,\u0020]')
chinses_english_number_s_unicode_space = re.compile(r'[^\u4e00-\u9fa5^a-z^A-Z^0-9]')
number_s_unicode_space = re.compile(r'[0-9]')
english_unicode_space = re.compile(r'[a-z^A-Z]')



def drop_2_to_1_dim(mylist): 
    return [i for j in range(len(mylist)) for i in mylist[j]]

def spread(arg):
    ret = []
    for i in arg:
        if isinstance(i, (list,tuple)):
            ret.extend(i)  
        else:
            ret.append(i)
    return ret

def deep_drop_dim(List):
    result = []
    result.extend(
        spread(list(map(lambda x: deep_drop_dim(x) if type(x) in (list,tuple) else x, List))))
    return result

def is_english_char(character):
    #验证是否为英文
    ordchar = ord(character)
    if ordchar not in range(97,122) and ordchar not in range(65,90):
        return False
    else:
        return True
    
def is_chinese_char(character):
    #验证是否为中文
    conditions = lambda character , fist ,end :character >= fist and character<= end
    if conditions(character, u'\u4e00',u'\u9fa5') or conditions(character, u'\u9fa6',u'\u9fcb'):
        return True
    else:
        return False

def split_english_and_other_start(sentence):
    # 分割英文
    words_list = []
    mark_point_true = -1
    if isinstance(sentence,(float)):
        return sentence
    for char in sentence:
        test_char = is_english_char(char)
        if test_char != True:       
            words_list.append(char)
        else:
            mark_point_true += 1
            words_list.append('/')
            words_list.append(char)
            if mark_point_true > 0:
                words_list.pop(-2)            
    return ''.join(words_list)

def split_chinses_and_other(sentence):
    # 分割中文与其他字符
    mark_list = [ord(sentence[0])]
    char_list = []
    mark_point = 0
    if isinstance(sentence,(float,int)):
        return sentence
    for char in sentence:
        test_char = is_chinese_char(char)
        mark_list.append(test_char)
        mark_end2 = mark_list[-2:]
        mark_test = mark_end2[0]-mark_end2[1]
        if mark_test != 0:
            char_list.append('/')
            char_list.append(char)
        else:
            char_list.append(char)
    mark_list.clear()
    return ''.join(char_list)


def split_space(text):
    #分割空格
    return re.split(r'(?P<space>\s)',text)

def split_to_phrase(text):
    #浅段落分割
    return list(filter(lambda x : len(x) > 1,re.split('r[^\u4e00-\u9fa5^a-z^A-Z]',text,flags = re.IGNORECASE)))

def deep_split_phrase(text):
    reLambda = filter(lambda x : len(x) > 1,re.split('r[^\u4e00-\u9fa5^a-z^A-Z^d]',text,flags = re.IGNORECASE))
    #深度段落分割 ：分割段落，中/英文，中文/数字
    return drop_2_to_1_dim([[deep for deep in split_chinses_and_other(shallow).split('/') if deep != ""] 
            for shallow in 
            reLambda])

def search_cutting(article):
    # 全文分词搜索引擎模式
    deep_split_string = ''.join(["{}{}".format('/',pharse) for pharse in article])
    return [i for i in jieba.cut_for_search(deep_split_string,HMM=True) if i != '/']

def cutting(article):
    # 全文分词精确模式
    deep_split_string = ''.join(["{}{}".format('/',pharse) for pharse in deep_split_phrase(article)])
    '''对i进行英文分词'''
    return [i for i in jieba.cut(deep_split_string,cut_all=False,HMM=True) if i != '/']

def all_cutting(article):
    # 全文分词全模式
    deep_split_string = ''.join(["{}{}".format('/',pharse) for pharse in deep_split_phrase(article)])
    return [i for i in jieba.cut(deep_split_string,cut_all=True,HMM=True) if i != '/' and i != '']

def word_probability(allWordList,wordList):
    ## 词概率计算 allWordList<-整个的词频列表,wordList<-要比较计算的词频列表
    set_word = set(allWordList)
    m = len(wordList)
    return {word:wordList.count(word)/m if word in allWordList else 0 for word in set_word}

def word_frequency(allWordList,wordList):
    ## 词频计算 allWordList<-整个的词频列表,wordList<-要比较计算的词频列表
    set_word = set(allWordList)
    m = len(wordList)
    return {word:wordList.count(word) if word in allWordList else 0 for word in set_word}


#def word_frequency_of_article(article):
#    ## 一篇文章本身的词频
#    cut = cutting(article)
#    return word_frequency(cut,cut)

def to_ndarray(data):
    # list tuple pandas 转 ndarray
    if isinstance(data,(np.ndarray)):
        data = data
    else:
        data = np.array(data)
    return data

def Hx(X,x,In='bit'):
    ## 计算单词的信息熵
    X ,x = to_ndarray(X),to_ndarray(x)
    n = X.shape[0]
    px = np.dot(np.array(X==x).astype(int)/n,np.ones(n))
    if px == 0 :
        return px*0
    if px == 1:
        return 0
    else:
        if In == 'bit':
            return -px * math.log(px,2)
        elif In == 'nat':
            return -px * math.log(px,math.e)

def HXi(article,In='bit'):
    #计算单篇全文的信息熵
    X = to_ndarray(cutting(article))
    n = X.shape[0]
    PXi= np.array([Hx(X,x,In) for x in X])
    PX = np.dot(PXi,np.ones(n).T)
    return {'entropy':PX,'probability':{x:p for x,p in zip(X,PXi)},'density':PX/PXi.shape[0]} 

def sorted_dict(dict_structure):
    #字典排序
    return sorted(dict_structure.items(), key=lambda x: x[1], reverse=True)

def emotion_list(text):
    # 情感分析向量list结构
    emotion = Emotion()
    emotiondict = emotion.emotion_count(text)
    emotionkeys = ('words', 'sentences', '好', '乐', '哀', '怒', '惧', '恶', '惊')
    return [emotiondict[i] for i in emotionkeys]

def emotion_dict(textList):
    # 多文本情感分析，输入list(str,str)，输出字典结构，可直接转dataframe
    emotionData = np.array([emotion_list(words) for words in textList])
    words,sentences,good,joy,sorrow,anger,fear,evil,fright = [],[],[],[],[],[],[],[],[]
    for values in emotionData:
        words.append(values[0])
        sentences.append(values[1])
        good.append(values[2])
        joy.append(values[3])
        sorrow.append(values[4])
        anger.append(values[5])
        fear.append(values[6])
        evil.append(values[7])
        fright.append(values[8])
    return {"words":words,"sentences":sentences
     ,"good":good,"joy":joy
     ,"sorrow":sorrow,"anger":anger
     ,"fear":fear,"evil":evil
     ,"fright":fright}


def words_of_deleted(word):
    stop_words = [line.strip().decode('utf-8') for line in open('/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/stop_word.txt','rb').readlines()]
    posseg = dict(jieba.posseg.cut(word)) # 词性标注
    flag =posseg[list(posseg)[0]]
    if flag in ['r','uj']:
        return None
    elif word  in stop_words:
        return None
    else:
        return word

def split_words_from_paragraphList(paragraphList):
    #清理文章列表的单词也可以文本以list形式包裹得到单个文章的删除了不需要的词的单词
    def clearn(paragraph):
        reLambda = re.split(r'[^\u4e00-\u9fa5^a-z^A-Z]',paragraph)
        words = deep_drop_dim([cutting(short) for short in 
                           list(filter(lambda x : len(x) > 1
                                       ,reLambda))]) 
        return words
    all_words = deep_drop_dim([clearn(paragraph) for  paragraph in paragraphList])
    return [words for words in all_words if words != None and words_of_deleted(words) != None]#删除一部分不要的词 

def split_words_from_paragraph(paragraph):
    #清理一篇文章，去除符号和不需要的单词
    #print('LLLL',paragraph)
    reLambda = re.split(r'[^\u4e00-\u9fa5^a-z^A-Z]',paragraph)
    words = deep_drop_dim([cutting(short) for short in 
                           list(filter(lambda x : len(x) > 1
                                       ,reLambda))])
    return [word for word in words if word != None and words_of_deleted(word) != None]#删除一部分不要的词 

def frequency_dictionary_editor_for_paragraphList(paragraphList):
    #文章总列队计算（频率/特征）字典编辑器 
    #传入文章列表
    #返回字典结构的词与目前出现的词频
    all_words_data = split_words_from_paragraphList(paragraphList)
    words_list = sorted_dict(word_frequency(all_words_data,all_words_data))
    return {element[0]:element[-1] for element in words_list }

def frequency_dcitionary_editor_for_paragraph(paragraph):
    # 单文章特征字典编辑器
    word_frequency_list= split_words_from_paragraph(paragraph)
    #print(word_frequency_list)
    return word_frequency(word_frequency_list,word_frequency_list)


def words_vector(feature_dict_,paragraph):
    #单文章字符串与特征字典比较得到的词频向量
    one_paragraph_dict = frequency_dcitionary_editor_for_paragraph(paragraph)
    feature_words = feature_dict_.keys()
    paragraph_words = one_paragraph_dict.keys()
    return np.array([one_paragraph_dict[word] if word in paragraph_words else 0 for word in feature_words])

def cosl(feature_vector,vector):
    # 计算文章词向量与特征向量的余弦值
    return vectorial_angle_cosine(feature_vector,vector)

def vectorial_angle_cosine(vector1,vector2,flag = 'cos'):
    ## 向量夹角余弦
    ## 余弦相似系数等价于归一化的person相关系数
    vector1,vector2 = to_ndarray(vector1),to_ndarray(vector2)
    if len(vector1.shape) | len(vector2.shape) | 1:
        cosangle = vector1.dot(vector2)/(np.linalg.norm(vector1) * np.linalg.norm(vector2))
        cos = cosangle*2*np.pi
        radian = cosangle*(np.pi/180)
        return np.array([cosangle,cos,radian])#{'cosangle':cosangle,'cos':cos,'radian':radian}[flag]
    else:
        return None
        
checkLambda = lambda List : set([word for word in split_words_from_paragraphList(List) if len(word)<2])

def connect_the_dict(dict_of_fist, dict_of_second): 
    dict_of_second.update(dict_of_fist)
    return dict_of_second

def words_code_dict(infoList):
    #词向量编码
    feature_dict_ = frequency_dictionary_editor_for_paragraphList(infoList)
    words_list = [i for i in feature_dict_.keys()]
    feature_vector = np.array([i for i in feature_dict_.values()])
    code_vector_dict = {words_list[i]:words_vector(feature_dict_,jobs_info_List[i]) for i in range(len(infoList))}
    return code_vector_dict

def words_code_mat(infoList):
    #词向量编码
    feature_dict_ = frequency_dictionary_editor_for_paragraphList(infoList)
    words_list = [i for i in feature_dict_.keys()]
    feature_vector = np.array([i for i in feature_dict_.values()])
    code_vector_mat = np.array([words_vector(feature_dict_,jobs_info_List[i]) for i in range(len(infoList))])
    columns=[word for word in words_list]
    return code_vector_mat,columns

def create_words_vector(tag,info_List):
    #词向量表
    mat,names = words_code_mat(info_List)
    newnames = ["{}{}{}".format(tag,'-',name) for name in names]
    words_table = pd.DataFrame(mat,columns=newnames)
    return words_table

def get_cos_dicts(Iterable):
    #文本余弦计算表函数，输入list set ndarray pandas tuple 输出 dict 可直接转pd.dataframe
    time_start=time.time()
    algorithm_of_post = list(Iterable)
    
    time_step1=time.time()
    step1 = (time_step1-time_start)*60/60
    print('the first step need , list text and check it',step1)
    
    feature_dict_ = frequency_dictionary_editor_for_paragraphList(algorithm_of_post) #特征字典计算
    time_step2=time.time()
    step2 = (time_step2-time_start)*60/60
    print('the second step need , create feature dict',step2)

    feature_vector = np.array([i for i in feature_dict_.values()]) #特征向量
    time_step3=time.time()
    step3 = (time_step3-time_start)*60/60
    print('the third step need , create feature vector',step3)
    
    #word_vector_dict = [len(words_vector(feature_dict_,algorithm_of_post[i])) for i in range(len(algorithm_of_post))]
    #print(word_vector_dict)
    word_vector_mat = np.array([words_vector(feature_dict_,algorithm_of_post[i]) for i in range(len(algorithm_of_post))])
    cosmat = vectorial_angle_cosine(word_vector_mat,feature_vector)                                                                                        
                                                                                            
                                                                                            
    #all_cos_dict = {i:cosl(feature_vector,words_vector(feature_dict_,algorithm_of_post[i])) for i in range(len(algorithm_of_post))}
    time_step4=time.time()
    step4 = (time_step4-time_start)*60/60
    print('The fourth step need ,get all words dict cosangle ',step4)
    print({'count':len(algorithm_of_post),'list text':step1,'statistical frequencyy':step2-step1,'create frequency vector':step3-step2,'cosl':step4-step3})
    
    '''整理成字典'''
    
    #cos_filter = lambda flag : [all_cos_dict[cosd][flag] for cosd in all_cos_dict]
    #cos_vector = cos_filter('cos')
    #cosangle_vector = cos_filter('cosangle')
    #radian_vector = cos_filter('radian')
    cos_dict_for_table = {'cos':cosmat[0],'cosangle':cosmat[1],'radian':cosmat[2]}
    return cos_dict_for_table

def create_stats_word_table(name,info_List):
    #文本特征统计表
    dict_ = eigenvector(info_List)
    table = pd.DataFrame(dict_)
    column_names = ["{}{}".format(name,i) for i in table.columns]
    table.columns=column_names
    return table

def get_entropy_dict(paragraph_List):
    #获得文本列表的信息熵字典 输入list(strings0,strings1...) 输出 dict 可直接转dataframe
    HXiList = [HXi("".join(["{}{}".format("/",i) for i in split_words_from_paragraph(paragraph)])) for paragraph in paragraph_List]
    entropy =[H['entropy'] for H in HXiList]
    density =[H['density'] for H in HXiList]
    return {'entropy':entropy,'density':density}

def eigenvector(info_List):
    #统计文本列表的余弦特征，生成特征词向量，情感特征，信息熵特征
    # 输入list（strings0，strings2...）输出 dict
    word_cos_dicts = get_cos_dicts(info_List)#余弦相似度特征统计
    cos_table =pd.DataFrame(word_cos_dicts) 
    word_emotion_dicts=emotion_dict(info_List)#情感特征统计
    emotiontable = pd.DataFrame(word_emotion_dicts)
    info_entropy_dicts = get_entropy_dict(info_List)#信息熵统计
    eigenvector_dict = connect_the_dict(info_entropy_dicts,connect_the_dict(word_emotion_dicts,word_cos_dicts))
    eigenvector_table = pd.DataFrame(eigenvector_dict)
    return eigenvector_table

In [3]:
info_List = ["'岗位职责：1、负责对游戏运营的相关数据进行分析，完成从数据埋点、数据接入、验证、分析至数据可视化、专项数据分析报告输出的全套数据分析工作；2、迭代优化固定数据分析产品，提高分析工作的时效性、准确性、稳定性、灵活性；3、负责与产品、客户端、后台、测试沟通协调，并基于数据分析结果推动问题解决或游戏优化，持续关注后续数据表现及玩家反馈；任职资格：1、性格活泼开朗，态度积极向上，富有钻研精神；2、坚实的统计基础知识，能够灵活运用统计工具解决实际问题；3、熟练掌握●SQL，能够实现数据拉取整合等基础操作；熟练掌握●R，能够实现数据整合、计算及结果展示;4、良好的沟通表达能力，能够使用简单通俗的语言准确、快速传递分析结果；5、缜密的逻辑思维，能够透过复杂的现象敏锐捕捉到事务的本质；6、统招本科及以上学历，统计、机器学习相关专业，经验不限，英文能力强者优先。【福利待遇】1.●关注你的职业发展提供有竞争力的薪资，每年稳定涨薪，高达10个月的年终奖、额外经营奖，更有导师制度和任职资格体系，帮助你应对职业发展未来的挑战。2.为你提供全方位保障为你提供六险一金和深圳户籍，超长的带薪假期，新婚添丁津贴和各类慰问金，每年定期体检，成为你工作打拼的有力保障。3.和你一起享受生活零食水果下午茶、入职生日节日礼包，各种生日会、团建、社团、周年庆活动，一年两次海内外旅游，工作之余，我们更想和你享受生活。\\xa0【关于我们】深圳市乐易网络股份有限公司，成立于2011年，公司目前规模200~300人。我们是一家专注海外手机游戏自主研发和自主运营的企业，致力于打造世界精品手游。\\xa0【主推产品】●Blaze●of●Battle一款超经典的●SLG游戏！在这里，你可以建立宏伟的城市，组建强大军队，召唤巨龙与骑士和您一起征服战争之光的战场。加入联盟，与来自世界各地的数百万玩家并肩作战，体验战争的刺激！\\xa0●West●Game一款人气爆棚的西部主题●SLG游戏。发行之初便屡次闯入北美i●OS畅销前一百。在狂野又刺激的西部世界中，你可以建立自己的小镇、创建你的帮派，和来自世界各地的玩家们战斗。\\xa0●Top●Gang一款主打黑手党题材的卡牌游戏。这是一个混沌和堕落的世界，衰落的秩序和廉价的道义，在地下世界中你将如何抉择——夹缝求生？又或是成为规则的主人？快来加入黑手党的世界，招募英雄，升级技能，一起战斗吧！'",
 "'1、参与●BI相关大数据系统的需求对接、架构设计、开发工作；2、参与需求构建数据仓库及数据集汇总优化，协助后台技术团队完成●DW、●ETL功能实现及相关●IT项目；3、对●BI用户数据分析技术支持、●IT技术支持、用户培训和平台运营提供数据支持。4、针对业务部门提出的各维度数据问题，进行数据获取，整理和分析，得出合理的结论，并以报告的形式呈现；5、深入发掘运营业务中的异常数据，主动分析并提供结论，推进业务部门调整及优化；6、负责规划和实施中对各业务主题进行分析及优化；7、开发和维护系统平台与框架，发现和解决存在的技术问题，保证●BI系统的性能和稳定性；8、负责项目级（复杂）报表需求进度，参与实施、推进项目达成验收；9、支持业务部门对业务数据进行挖掘，找出核心问题，并提出关键举措，推动运营指标及业务能力的提升级。任职要求：1、本科及以上学历，计算机科学与技术、统计学、数学、数据科学、财务会计等本科及以上学历；2、3年及以上的●BI项目及数据分析工作经验；3、熟练使用至少一种关系型数据库（●Oracle,●Sql●Server,●My●Sql），有数据库优化经验，有非关系型数据库使用及●PL/●SQL等脚本编程经验优先；4、对数据敏感，具备优秀的分析技巧以及报告展示能力，了解主流报表工具或新兴前端报表工具；5、熟悉电商平台企业的●ERP中台、电商●EC\\\\●OMS、●WMS、财务金蝶等应用系统操作更佳；6、有管理和分析用户需求的经验，有平台商城各行业数据分析和数据洞察的技术支持经验优先；7、有较强的计划和组织协调能力，具备结果导向思维，工作踏实、对技术工作有热情。'",
 "'欢迎一起，寻找母星。希望多年后，回忆这段酣畅淋漓的创业历程，你不会后悔。岗位职责：1.●根据不同的业务场景，构建业务指标体系，建立和完善日常业务报告体系，能够及时、准确、完整的披露项目的运作情况；2.●通过专项分析，输出专项分析报告，为业务模块的决策和产品方向提供数据支持；3.●能够持续动态的挖掘日常的业务数据，发现业务的风险和机会点。\\xa0岗位要求：1.●本科或以上学历，理工科专业；2.●有互联网行业大规模数据分析、产品改进迭代相关工作经验；3.●具备编码能力，精通●SQL，熟练应用●Python/●PHP等脚本语言，能独立完成相关的数据分析工作；4.●具备良好的沟通交流能力和文字语言表达能力，较好的逻辑分析能力；5.●有和产品、运营一同有过产品周期运营，并协助产品、运营分析和改进产品功能者优先。**********************************************【寻找母星，是谁】寻找母星是一个互联网基因强大的创业团队——●CEO，共享充电宝「街电」创始人；联合创始人，百姓网首席架构师，nonda平台总监，食色创始人；●CTO，阿里云早期创始团队出身，周末去哪儿网●CTO；商务团队铁军云集，美团点评深圳负责人、去哪儿网广深负责人、猫眼电影华东大区总●……18年6月成立。天使轮已获得美金机构千万级融资，估值过亿。'",
 "'工作职责：1、●负责合作客户数据测试，灵活解决客户测试过程中遇到问题；2、部门内部数据报表所需数据的处理与准备；3、基于部门现有的基础数据，挖掘有价值的公共数据。岗位要求：1、本科及以上学历，1年以上数据分析经验；2、熟悉数理统计及数据分析常用算法，对数据敏感，能从海量数据中提炼出核心结果；3、具有数据处理和分析经验，并能熟练使用●SQL、●Python等软件进行数据分析；4、具备良好沟通能力和抗压能力。'",
 "'职位描述：●1.●负责广告投放分析，持续跟踪投放效果；●2.●深入研究极光大数据，分析解读业务需求，挖掘广告投放目标人群；●3.●协同广告运营团队，在贷款、游戏、教育等多个行业的投放过程提供及时有效的数据分析支持以及解决方案；●4.●定期沉淀及梳理广告投放过程中的经验和问题，不断在新行业投放中取得突破。职位要求：●1.●本科及以上学历，数学、统计学专业；●2.●具有两年以上与数据分析相关工作经验，具有广告投放分析经验者优先；●3.●熟练使用●SQL/●HQL数据库查询语言，有相关项目经验；●4.●熟悉python或●R，有数据处理，统计分析，数据建模，机器学习等相关算法应用经历；●5.具有良好的逻辑思维能力和数据敏感性，具有良好的多任务管理和项目推动能力。'",
 "'岗位职责：1、数据预处理：数据爬虫、地图数据编辑、校正以及日志●ETL清洗2、协助●BI做数据准确性测试以及协助●BI报表定时任务以及数据可视化的开发3、跟随产品优化、项目进度，对应产出改进或新建数据分析报告，协助●BI产品经理发现问题；4、解读关键性业务指标，结合数据和业务定位关键问题，产出日常报表；5、日常数据提取和分析，满足其他业务方数据分析需求。任职要求：1、计算机、统计、数据挖掘、●GIS、交通相关专业；2、熟练掌握●Python完成数据清洗、数据爬虫、数据可视化以及数据接口开发；3、有数据挖掘相关经验，掌握各种常见机器学习算法，对深度学习有一定了解优先；4、能使用●Power●BI或●Smart●BI（或任一种常见●BI工具：●QV，●Tableau，●Quick●BI）制作和发布数据报表优先；5、数据敏感度高，逻辑分析能力强，学习能力强，耐心细致有良好的沟通能力和团队协同工作能力；6、了解●R、●Node●JS、●Java、●Mongo●DB、●Spark●SQL或其它任一种数据工具/语言优先公司介绍：滴滴优点科技（深圳）有限公司于2016年3月由深圳巴士集团、滴滴出行及中科院深圳北斗院三方共同投资成立。滴滴优点基于大数据、物联网和移动互联网技术，专注于智慧公交调度系统和公交共享出行平台研发。“优点出行”、“优点调度”产品已在全国多个城市上线运营。'",
 '0',
 "'岗位职责：1、负责数据提炼、分析、归纳用户属性、行为等信息，基于大数据分析，提供用户画像，对各类事件进行分析和规则匹配，建模应用场景；2、独立完成后台相关数据分析报告及个性化需求数据分析，完善数据报表体系，及时准确监控运营状况，并提供专业分析报告；3、负责大数据运营平台数据服务产品的规划设计；4、深入理解业务模型和未来趋势，从对产品、业务和商业的分析挖掘中寻找数据商业价值点，从整体的角度寻找数据的商业价值和机会。任职要求：1、本科及以上学历，计算机、统计、经济、数理、信息等相关专业；2、具备1-2年的●ETL开发经验，熟悉●Java开发，熟练使用●Kettle、●Informatica等●ETL工具；\\xa03、熟悉任意一种数据库软件（●Oracle、●Mysql、hive等），具有良好的●SQL编写与优化能力；4、熟悉数据库设计，了解数据建模，能建立合理高效的数据模型者优先；5、熟练使用axure、墨刀等原型工具，能够独立设计输出数据服务；6、加分项：掌握分类、回归分析、聚类、关联规则等数据统计模型，具有独立完整的建模实践经验，能够根据数据报表进行业务建模，做相关数据预测工作。'",
 '0',
 "'职责描述：1、负责公司营销业务数据分析，包括渠道数据或报告分析等；2、负责各种数据跟踪平台的数据处理方式，布局公司所有站点的跟踪代码；3、针对业务部门提供的数据问题进行数据获取、整理和分析，得出合理的结论并以报告形式呈现；4、深入发掘运营业务中的数据异常，主动分析并提供结论，推进营销业务调整和优化。任职要求：1、本科及以上学历，数学、统计及电商等相关专业；2、三年以上数据分析工作经验，熟悉数字营销行业运营工作，能独立完成过商业数据分析项目；4、熟练使用一种及以上专业数据分析工具和数据库查询操作，熟练使用●GA系统；5、责任心，主动性强，有良好的沟通能力和团队合作精神；6、大学英语六级，熟练使用office办公软件等。'",
 "'职位诱惑：14薪；年轻有活力的团队；五险一金；职位描述：岗位职责:1.深入理解业务，通过数据分析、挖掘、建模，设计产品和业务逻辑，解决业务核心痛点；2.参与建立和优化包括风控模型在内的多个模型：依托数据仓库进行数据的提取、整理，按照标准数据挖掘流程进行数据的清洗、抽样、建模和评估；3.发挥数据引领业务的作用，参与公司金融科技类产品的研发和落地实施；4.负责产品和模型的运营分析、跟踪评价、持续优化，通过数据全面反映产品的运营状态和问题；任职资格:1.统计学、数学、计量经济学、计算机等相关专业，本科及以上学历；2.具备营销建模能力3.熟悉数据仓库知识，熟悉●Hadoop数据库，掌握●SQL,●Hive等查询语言；4.熟悉并至少熟练掌握一种统计分析语言，包括但不限于●SAS、●R、python等；5.掌握常用的统计预测模型、数据挖掘模型，具有较强的分析建模能力；6.良好的沟通表达、协调整合能力；7.较高的责任心、担当能力；8.一年以上金融相关行业（行业从业或相关项目经验）工作经历，有银行数据分析经历者优先。9.此职位有出差佛山需求'",
 "'岗位职责：1、深入理解互联网电视广告、内容、会员等业务，系统建立和完善日常业务监控体系，呈现业务经营情况。2、监测分析业务运营状况，对异常波动进行分析，持续优化产品功能改造和业务拓展方向，提出和落实产品优化改造方案。3、分析用户行为数据，探索用户行为模式，构建用户生命周期和构建用户标签体系等。4、完成不定期的专题分析报告，并做演示汇报。任职资格：1.●本科及以上学历，信息、管理科学、统计、应用数学等相关专业优先；2.●能熟练运用●SQL、excel，至少掌握一种数据分析工具●SPSS、python、●R等，独立且高效地完成数据的提取及分析，有处理海量数据的经验；3.●有数据敏感度，能从海量数据中提取核心结论，并用清晰且简洁的方式呈现其商业洞察；4.●有互联网行业经验，●OTT行业经验优先；5.●逻辑思维清晰，严谨细致，有良好的沟通理解能力和跨团队协作精神；'",
 "'岗位职责：1、负责数据分析计算平台的需求分析、架构设计、编码实现；2、负责具体项目的数据处理算法设计、编码实现、持续改进；3、负责对海量业务数据进行处理和分析，包括数据收集、清洗、整理、存储、挖掘分析等工作。任职要求：1、本科或以上计算机或统计类相关专业；3年以上数据分析相关工作经验；2、掌握数据分析的原理与方法，熟练使用数据分析工具和开发库；3、精通●Java、●Python或●C++至少一种编程语言；4、精通数据库操作，●SQL语言编写，数据库存储过程编写；5、有数据可视化、报表系统的开发经验；6、良好的业务分析能力'",
 "'岗位职责：1、深入了解互联网业务，建立基于业务场景的数据分析需求，解决各类数据分析问题；2、基于客户各业务部门的分析监控需求，搭建指标体系，并设计数据采集的埋点方案，实现数据可视化；3、根据客户需求，制定出最佳的数据分析策略，产出行业分析案例，建立数据分析的流程、规范和方法；4、基于数据分析，与销售、服务、产品、工程团队以及其他主要相关方开展跨职能协作，助力相关产品/服务取得成功。岗位要求：1、本科以上学历，经济学、统计学、计算机等和数据处理高度相关专业，3年及以上互联网公司数据分析工作经验；2、具备杰出的定量分析能力，在推动业务/产品运营优化方面拥有丰富的经验；3、具有优秀的商业敏感度和突出的分析能力，能够灵活利用数据，设计分析方案解决复杂的商业问题；4、擅长与内部及外部合作团队交流沟通，具有优秀的报告讲解及沟通能力，能够独立的分析和解决问题；5、有数据建模实践经验（2年以上相关工作经验）者优先，熟悉●Hive，熟练使用●SQL等数据库查询语言；6、具有较强结构化思维、逻辑思维能力，对数据敏感，具备优秀的信息整合和分析能力，能够形成清晰的业务观点和前瞻判断。'",
 "'岗位要求：1、本科及以上学历，2年以上互联网工作经验；2、对数据敏感，了解数据分析的方法；学习能力强，有足够的细心与耐心；3、具备良好的逻辑思维能力与表达能力，良好的产品推动能力和团队协作能力岗位职责：1.●负责华为应用市场数据需求管理，参与数据需求设计及评审、试算到上线全流程；2.●监控业务数据，对数据异常通过分析形成需求和改进方案；3.●通过数据梳理、数据处理、数据分析、数据可视化形成数据分析报告。'",
 "'【岗位职责】1、负责数据平台的日志数据采集、数据传输、数据存储等数据相关的规划工作，并且对埋点采集的数据质量、应用负责；2、深入了解各业务线的数据，能解释数据背后的业务含义，从中发现问题，并建立相关的数据报表及预警机制；3、能独立编写专题分析报告，阐述数据过程，提出问题和建议，有以终为始的数据驱动业务的意识。【岗位要求】1、全日制本科及以上学历，计算机、统计学、数学等相关专业，三年以上互联网公司数据分析相关工作经验；2、能熟练运用●SQL、hive、excel等工具，独立且高效地完成数据提取、清洗、分析、展示等工作；3、对数据采集、埋点、清洗入库、分析应用有系统的认知；4、工作积极主动，有责任心，有较强的逻辑思维能力、沟通表达能力和自我学习的意识。【薪酬福利】年14薪+●N（人均16薪及以上）；●每年一次加薪机会，年薪平均涨幅20%以上●；差异化购买公积金政策；员工互助金政策（员工意外伤害保障）【工作地点】福田八卦岭八卦路【交通便利】1.距7、9号地铁口八卦岭站、红岭北站步行5分钟\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●\\xa0●2.公交枢纽：泥岗村、八卦路'",
 "'【岗位描述】【岗位要求】'",
 "'岗位职责：●1.●主导增长数据分析项目；●2.●根据增长业务节奏，给出数据支持节奏，有力支持业务；●3.●从需求出发，推动上报，中间表，系统数据能力落地；●4.●建设指标监控体系，有效解释数据异动；●5.●建设增长数据系统，提供系统化的数据分析能力，满足业务需求；●5.●定期输出数据分析报告，提出产品/推荐/增长/内容等方向的优化点；岗位要求：●1.●本科及以上学历；●2.●有2年以上互联网行业数据工作经验；3.扎实的数据开发和编码技能●；4.有过数据治理、数据平台、数据分析经验；●5.●很好的业务和产品理解能力；●6.●熟练使用●SQL/●HIVE/●SAS/●SPSS/●R/●Python;'",
 "'【岗位职责】1、深入了解公司业务，与各团队有效沟通，收集和整理数据分析需求；2、对流量、用户、交易、渠道、活动等数据进行定期统计和分析，输出业务洞察报告；3、配合产品经理进行相关产品及项目的定型，完成数据埋点、建模、分析维度设计等；4、基于各部门的分析监控需求，搭建指标体系，并设计数据采集的埋点方案，实现数据可视化；5、根据数据分析和对业务形态的理解，对产品、服务、营销等活动提出合理化建议。【岗位要求】1、本科以上学历，经济学、统计学、计算机等和数据处理高度相关专业，一年及以上互联网公司数据分析工作经验；2、具备杰出的定量分析能力，在推动业务/产品运营优化方面拥有丰富的经验；3、具有优秀的商业敏感度和突出的分析能力，能够灵活利用数据，设计分析方案解决复杂的商业问题；4、擅长与内部及外部合作团队交流沟通，具有优秀的报告讲解及沟通能力，能够独立的分析和解决问题；5、具有较强结构化思维、逻辑思维能力，对数据敏感，具备优秀的信息整合和分析能力，能够形成清晰的业务观点和前瞻判断。6、具备较强的数据处理能力，以及●Excel、●PPT处理能力；【你将额外获得】1.该职位直接和●CEO汇报，将获得知名网红●CEO@老曹亲自指点；2.表现出色，可获得起点学院课程免费学习机会，逆天福利！！！3.年轻化团队好融入，以做事为中心，是学习成长的绝佳环境；4.下午茶和水果，不定期室内室外各种团建活动；5.有机会接触●BAT等头部公司大咖，积累高端人脉甚至获得指点；【有以下习惯的小伙伴不建议投递该职位】1，需要有人保姆式的带教和监督、跟进，无法自觉独立完成工作的；2，学习能力较差，不关注同行竞品；3，主动性差，遇到问题不会主动推动解决；4，执行能力差，做事经常没反馈，经常没下文；'",
 "'岗位职责：\\xa01.结合业务需求，搭建相应数据分析体系，建立监控模型，支持产品数据化运营；2.利用数据分析和数据可视化，从数据收集/准备到模型实现和商业洞察，开展数据管理和分析；3.规划和建设基于接入机构及内部相关管理部门的数据主题，有效地向各业务团队传达解决方案和分析结果，推动业务优化；4.研究大数据领域前沿数据分析挖掘技术，对前沿技术结合产品业务进行实验及落地；\\xa0\\xa0岗位要求：\\xa01.●本科及以上学历，计算机、数学相关专业，三年及以上工作经验，在风控、路径规划等相关领域有较多的算法工作经验；2.●优秀的逻辑思维能力以及分析和解决问题的能力；具有良好的数据敏感度，能够从海量数据中发现有价值的规律；\\xa03.●熟悉常用的机器学习和数据挖掘原理与算法，如分类、聚类、回归、协同过滤等；对常用算法如●ARIMA、●LR、●GBDT、●SVM、●XGBOOST等任意一种的模型原理以及工程实践上有较多经验；4.●至少熟悉java/scala/python中的一种语言，至少熟悉一种常用的机器学习库，具有扎实的编程基本功；\\xa05.●具有良好的团队合作精神和较强的沟通能力；善于思考和学习，对算法落地有充分的理解，对有挑战性的问题充满激情。'",
 "'【岗位职责】：1.负责腾讯指数产品的内容运营。负责内部自媒体平台（微信等）、官网、公众号、企业号等各个内容渠道运营建设和把控，建立和执行有效的内容管理机制；2.负责收集、研究网络热点话题，能结合热点通过数据分析、文章撰写进行产品宣传，协助腾讯指数品牌进行对内容策略的实时调整和内容的及时更新；3.策划并推动内外部的品牌合作活动，不限于讲座、●H5等推广形式，定期以拉新、粉丝增长活跃的数据有效提升来策划推广活动；4.分析运营数据，结合数据及用户反馈，有效调整运营策略，提高用户量、阅读量和互动数量。5.能承受一定的工作压力。【岗位要求】：1、具有较深厚的文字功底和文字驾驭能力，文笔流畅，并能够用合适的热点话题及软文风格对品牌进行包装；2、对主流自媒体平台内容分发及其它规则有较深了解，熟悉各种社会化媒体运营及渠道推广方法；3、善于与人沟通，快速执行力，有强大的自我驱动力。积极主动，执行力强，敢于接受挑战，性格开朗，有较强的团队合作精神；4、有一定数据分析基础，对数据敏感，懂得如何贴近热点，使用并解读产品数据，进行用户教育；同时，善于借助数据优化完善运营方案；5、熟练使用photoshop，以及word、excel等办公软件，能基础运用部分剪辑工具。'",
 "'岗位职责：1、对公司营销目标、业务增长负责，通过数据分析，探索业务机会点，输出目标拆解路径及增长方案；2、营销核心数据监控，挖掘过程中的异常情况与典型案例，挖掘可负责的增长方式；3、负责设计与搭建数据化用户运营体系，对用户进行全生命周期运营，通过数据对用户进行合理分层，设计不同层级用户的数据分析体系，持续提升营销转化；4、能独立完成数据分析报告，为营销方案、业务发展、运营决策提供数据支持及策略建议。任职要求：1、本科以上学历，5年以上数据分析与运营经验，有良好的商业sense；2、对用户体系有深入了解，具备资源整合能力，善于沟通协同；3、具有很强的数据化、产品化运营经验，能够设计数据分析与运营体系，善于从数据中发现问题并提出解决方案；4、具备较强的数据处理能力，以及●Excel、●PPT处理能力，●SQL数量者优先；5、有●BI、用户运营、业务数据分析经验，对房地产行业或企业信息化行业有一定了解优先。'",
 "'岗位职责：1.\\xa0\\xa0\\xa0\\xa0与各业务团队有效沟通，收集和整理数据分析需求，能快速理解相关业务；2.\\xa0\\xa0\\xa0\\xa0配合产品经理进行相关产品及项目的定型，前端交互以及分析维度设计，参与相关产品及项目的解决方案梳理和制定；3.\\xa0\\xa0\\xa0\\xa0对流量、用户、交易、渠道、活动等数据进行定期统计和分析，输出业务洞察报告；4.\\xa0\\xa0\\xa0\\xa0根据数据分析和对业务形态的理解，对产品、服务、营销等活动提出合理化建议。\\xa0任职要求：1.\\xa0\\xa0\\xa0\\xa0大专及以上学历，统计学、应用数学、计算机、计量经济、金融等相关专业；2.\\xa0\\xa0\\xa0\\xa0两年以上数据分析经验，具备较强的逻辑思维能力、决策判断力及较强的学习能力，沟通及表达能力强；3.\\xa0\\xa0\\xa0\\xa0熟练掌握和运用相关数据分析软件，了解线性回归模型，非线性回归模型，判别分析模型，聚类分析模型，因子分析模型，生存分析模型等；4.\\xa0\\xa0\\xa0\\xa0熟悉在线教育行业业务知识，善于输出洞察报告；5.\\xa0\\xa0\\xa0\\xa0熟练使用●Python、●R语言者优先。\\xa0'",
 "'岗位职责：1.●负责公司现有大数据产品（智子、数据治理、机器学习、背景库等）产品和方案的售前支持：负责与客户进行沟通交流，准确把握客户诉求和需求，对项目进行顾问式支持，并根据业务特点和公司产品情况提供有针对性的解决方案；\\xa0●\\xa02.出席各种市场与销售活动，提供解决方案宣讲，组织项目的招投标工作，促进项目成单率；\\xa0●\\xa03.总结典型客户应用，完成样板客户应用模式分析与价值提炼，并提供相关营销工具文档；\\xa0●\\xa04.面向客户、内部员工及合作伙伴的行业方案、案例及产品培训；\\xa0●\\xa05.理解目标市场，搜集市场信息，分析需求和行业趋势，参与产品策略的制定。\\xa0●\\xa0岗位要求：1．3年以上●IT售前、解决方案和大数据平台售前经验；\\xa0●\\xa02.了解大数据相关产品，熟悉招投标整个流程，具备很强的技术方案撰写和宣讲能力，有中、大型大数据项目售前经验；\\xa0●\\xa03.有较强的沟通能力和表达能力、抗压能力和团队合作意识，有很强的学习能力；\\xa0●\\xa04.本科以上学历，电子信息、通信、网络工程和计算机相关专业；\\xa0●\\xa05、熟悉公安领域者优先；\\xa0●\\xa06、能接受一定程度的出差。'",
 "'我们有：轻松舒适的办公环境，具有竞争力的薪资；五险一金，员工生日，佳节福利…一样都不能少！在这里，我们没有官僚教条主义；然而这些都不是我们最核心的竞争力。在这里，我们拒绝平庸；你可以天马行空，但不能眼高手低；你可以有个性，但必须有团队精神；在这里，你可以不完美不周全，但必须有想法有激情……我们有高端大气的射技狮，也有秋名山老司机测试喵，有沉稳闷骚的软件硬件攻城狮，也有激情不断的产品汪，他们是一群敲得了代码，搞得了设备，当得了麦霸，下得了厨房的小伙伴。小清新、文艺范、技术控、大叔范应有尽有，这里你会收获许多志同道合的最佳拍档！但是，我们离完美还有一点距离，因为我们还缺少你！来不及再说太多，抓紧时间上车了，●滴~岗位描述：1.●营销中心销售业务数据的分析整理归档，能对部门现有业务数据进行梳理汇总和跟踪监控，及时敏锐的发现业务数据变化趋势；2.●对业务数据进行整合、分析挖掘，挖掘产品以及用户潜在信息，为营销、运营及决策提供业务分析及数据支持。任职要求：1.熟练操作●EXCEL（会使用函数工具），●ERP系统等工具（金蝶等其他软件）；2.有良好的英语听说读写能力，有较强的逻辑思维能力（能主动思考）；3.抗压能力强，执行力强（先做，在想，后问）。4.本科及以上（专业方向是数学，财务，法律，审计等相关专业的同学优先考虑！）1年以上的相关工作经验（优秀的应届毕业生也欢迎尝试投递沟通）。'",
 "'1、负责海外各业务的数据相关工作，完成从数据埋点、数据接入、验证、分析至数据可视化、数据分析报告输出的全套数据分析工作；2、构建海外用户画像体系建设和精细化数据运营体系；3、负责以数据驱动的用户增长、竟品分析,●并以数据分析结论推动业务中的各个功能和流程的优化；4、根据业务发展需求，统筹设计业务指标与业务报表，并跟踪业务报表体系.\\xa0\\xa01、2年以上数据分析工作经验、专科及以上学历，数学、统计、计算机等相关专业●；2、对数据敏感，善于发现数据中的潜在规律，善于分析问题，有很强的逻辑思维能力和推理能力●;3、熟悉●SQL，熟悉python、●R、shell等至少一种语言●；4、参与过完整的数据采集、数据清洗整理、分析和建模工作者优先●；'",
 "'岗位职责：1、日常数据统计分析工作；2、负责国、地税数据和外部数据接入，标准化及转换；3、负责评分卡模型设计中的数据准备、加工清洗、指标设计等。岗位要求：1、本科及以上学历，数学、统计学及计算机相关专业。2、掌握python、●R、●SASS等任两种数据分析工具，熟悉●SQL脚本编写者优先；'",
 "'为项目外派驻岗，到深圳市罗湖区深业中心大厦上班（平安项目），介意慎投岗位职责：1、负责理解机器学习的算法及其适用场景，根据实际情况，研究适合的算法模型，并对模型进行持续调优；2、参与客户画像建模和数据挖掘工作，深入客户关系网、客户行为数据、特征挖掘、用户体系的建设；3、参与研究个性化推荐算法；任职资格：1、本科以上学历，计算机或相关专业；2、熟悉掌握●Python，●Linux；3、熟悉spark核心流程，两年以上spark和●Tensor●Flow等深度学习框架下的实际开发经验；4、了解常见的大数据开源组件使用及开发，包括但不限于●ES、spark、kafka等；5、精通数据结构、算法、机器学习模型；6、具有良好的团队合作及学习能力，善于沟通，工作自主驱动，具有良好的问题定位分析能力。'",
 "'岗位职责1、对公司运营进行定期统计和分析；2、协助技术部和产品部开发数据平台；3、通过数据挖掘对用户数据进行分析，为精细化运营及营销提供数据支持，提出合理化建议；4、定价规则的编写；熟悉●R语言；5、维护公司各种产品的价格以及及时更新；6、负责相关数据的收集、整理、统计和分析；7、负责产品数据分析工作，发现运营异常，分析原因，为决策者提供数据支持；8、领导临时安排的工作。\\xa0任职资格1、本科以上学历，统计学、应用数学、计算机、计量经济、金融等相关专业优先；2、熟练掌握和运用相关数据分析软件，至少掌握●SAS、●SPSS、●R、●MATLAB、●Excel等统计工具中的一种；3、具备较强的逻辑思维能力、分析判断力及较强的学习能力；4、熟悉●GA、百度统计等网站数据统计平台者，优先考虑。注：本岗位每周6天工作制，介意者勿投！'",
 "'岗位职责：一、日常工作1、数据的提取、查询、处理、分析等工作；2、产品数据分析：分析产品销售情况，建立数据模型，针对不同情况，提出合理建议；3、根据数据需求，提供数据模型、数据挖掘分析报告，对产品销售和优化提供数据支持；4、数据汇报：独立制作日/周/月报、线上线下●KPI分析报表等。二、数据体系建设1、出行平台数据报表的设计和维护；2、与技术开发团队沟通将原始数据导入到系统内，并确保数据运行逻辑的准确性3、出行业务数据指标体系建设，通过监控分析及时发现业务问题任职要求：1、本科及以上学历，2年以上电商●B2●B平台数据分析工作经验；2、办公软件精通，有过自动化●BI报表开发经验优先考虑；3、理解电商平台各类数据报表的设定逻辑，并能优化修正；4、良好的沟通能力，能够与技术开发团队完成日常沟通工作。5、能接受互联网电商行业的快速工作节奏。'",
 "'1、基于业务需求，规划分析思路，完成从数据提取、数据清洗、数据分析和报告产出的整个流程，为业务问题原因排查及解决方案提供数据支撑；2、在深入理解业务基础上，结合业务目标和问题，构建数据指标体系与分析模型，为数据化运营提供系统化的解决方案；3、主动分析并研究业务数据，基于数据分析结果，为管理层和业务部门提供商业策略分析和业务优化建议；4、基于业务分析经验，将关键分析思路进行产品化，并参与企业数据产品的设计与实现；5、通过研究和数据支持产品的优化改进，指导数据应用部门工作。1、统计学、数学、计算机信息类、数据挖掘等相关专业3年以上数据分析相关工作经验，具有互联网数据挖掘、用户行为分析，产业价值分析经验者优先；2、熟练使用●Excel、●PPT、●R/python、●SQL，有缜密的逻辑思维和较好的文字、数据、图表呈现能力；3、熟悉常用数据挖掘算法（如分类、聚类、回归等）及其原理，并具备相关项目经验；4、责任心强，细致认真，能承受较强的工作压力，热爱数据分析工作。'",
 "'岗位职责：1.●\\xa0建立并完善数据体系，制定整体业务数据获取规范、流程、标准，并监督执行、优化；2.●根据业务部门运营需求，梳理标准化数据指标，并建立、维护自动化报表体系，监控并反馈业务运营状况；3、深入理解业务及产品，跨部门协调和沟通，响应临时性数据分析需求；4、分析数据成因及业务运营关联性，发现问题并为企业决策提供支持。岗位要求：1、本科以上学历，统计、数学、计算机信息工程等专业优先；2、三年以上互联网行业工作经验，五年以上数据分析经验；3、具有扎实的统计学、数据分析、数据挖掘基础，熟悉数据分析、数据挖掘的手段；4、对数据敏感，归纳、分析能力强，做事严谨细致；5、有数据平台搭建等相关经验，有管理经验；6、精通office系列软件，熟练access、mysql等数据库软件，熟悉●SPSS、●Python、●MATLAB等软件。'",
 "'岗位职责：1.●\\xa0建立并完善数据体系，制定整体业务数据获取规范、流程、标准，并监督执行、优化；2.●根据业务部门运营需求，梳理标准化数据指标，并建立、维护自动化报表体系，监控并反馈业务运营状况；3、深入理解业务及产品，跨部门协调和沟通，响应临时性数据分析需求；4、分析数据成因及业务运营关联性，发现问题并为企业决策提供支持。岗位要求：1、本科以上学历，统计、数学、计算机信息工程等专业优先；2、三年以上互联网行业工作经验，五年以上数据分析经验；3、具有扎实的统计学、数据分析、数据挖掘基础，熟悉数据分析、数据挖掘的手段；4、对数据敏感，归纳、分析能力强，做事严谨细致；5、有数据平台搭建等相关经验；6、精通office系列软件，熟练access、mysql等数据库软件，熟悉●SPSS、●Python、●MATLAB等软件；'",
 "'欢迎一起，寻找母星。希望多年后，回忆这段酣畅淋漓的创业历程，你不会后悔。岗位职责：1.●根据不同的业务场景，构建业务指标体系，建立和完善日常业务报告体系，能够及时、准确、完整的披露项目的运作情况；2.●通过专项分析，输出专项分析报告，为业务模块的决策和产品方向提供数据支持；3.●能够持续动态的挖掘日常的业务数据，发现业务的风险和机会点。\\xa0岗位要求：1.●本科或以上学历，理工科专业；2.●有互联网行业大规模数据分析、产品改进迭代相关工作经验；3.●具备编码能力，精通●SQL，熟练应用●Python/●PHP等脚本语言，能独立完成相关的数据分析工作；4.●具备良好的沟通交流能力和文字语言表达能力，较好的逻辑分析能力；5.●有和产品、运营一同有过产品周期运营，并协助产品、运营分析和改进产品功能者优先。**********************************************【寻找母星，是谁】寻找母星是一个互联网基因强大的创业团队——●CEO，共享充电宝「街电」创始人；联合创始人，百姓网首席架构师，nonda平台总监，食色创始人；●CTO，阿里云早期创始团队出身，周末去哪儿网●CTO；商务团队铁军云集，美团点评深圳负责人、去哪儿网广深负责人、猫眼电影华东大区总●……18年6月成立。天使轮已获得美金机构千万级融资，估值过亿。'",
 "'岗位职责：1、通过数据分析方法，利用数据和算法深入挖掘用户价值；2、深入了解业务，和运营团队一起推动数据化运营，提供基于数据的决策；3、定期输出专项数据分析报告；4、参与用户画像体系建设，个性化推荐系统建设，风控系统建设；5、基于真实的用户行为数据，主动探索新的数据分析需求和业务场景。任职资格：1.本科及以上学历，计算机或数学相关专业。有数据分析、数据挖掘和数据建模经验者优先；2.学习能力强，对数据敏感；3.精通●HQL/●SQL，熟悉●Hive原理，了解常用机器学习算法；4.具备良好的分析解决问题能力，责任心强，良好的沟通、团队协作和自我驱动能力；5.在实际项目中使用过●NLP、分类、特征工程等算法，并取得一定成效者优先；'",
 "'...●....'",
 "'...●....'",
 "'工作职责：1、●负责风控反欺诈相关专题数据分析，基于分析提出策略优化建议；2、●负责风控反欺诈相关数据监控、数据异动分析；3、●负责风控反欺诈策略调整工作，优化当前反欺诈策略，推进策略落地并跟进上线监控。任职要求：1、●本科及以上学历，数学、统计、金融、经济等相关专业优先；2、●有金融行业数据分析、风控相关工作经验者优先；3、●掌握一种以上主流的统计分析工具：●SAS/●SPSS/●SQL/●R/●Python等；4、●工作主动细致、有耐心、学习能力强，能够承受较大的工作压力；5、●对数据敏感，具有较强文字表达能力及熟练的数据分析解读经验；6、●善于沟通、有良好团队协作意识，能快速融入团队。'",
 '0',
 "'岗位职责：1、负责游戏数据分析体系搭建，包括●TLOG埋点、日志上报、分析系统的规划等；2、负责游戏目标用户分析、数据预估、用户调研等，为运营计划及市场方案提供依据；3、负责游戏上线及版本更新时的效果分析，包括：留存及流失分析、付费分析、功能玩法分析、用户满意度调研等；4、通过数据监控游戏中的各种动向与产品健康度，及时发现不良运营情况。\\xa0任职要求：1、\\xa0数学、统计和计算机等相关专业本科及以上学历；2、3年以上游戏或互联网数据分析经验，数理逻辑清晰，数据敏感度高，能够独立完成游戏业务数据挖掘和分析工作；3、具备完整的数据挖掘、建模、预测、分析能力，熟练使用●SQL语句、●EXCEL；掌握●SPSS、●R、●SAS等其中一种分析工具；4、良好的沟通和协作能力，强烈的责任心，出色的学习能力和钻研精神；'",
 "'岗位职责：1.产品规划与设计。负责数据分析产品（●CEM客户体验管理）规划、业务流程设计、功能设计，制定阶段性目标；2.产品需求分析。制定产品需求计划，进行可行性分析及设计，撰写产品功能需求说明书，负责需求的跟踪和控制；3.竞品分析与用户反馈。把握市场前沿动态、研究并分析竞争对手、收集用户反馈，平衡业务发展和风控效率，保持产品竞争力；4.产品开发测试。主导产品的开发测试，协调各团队资源按期完成产品迭代；5.项目管理与推动。把握项目周期，跟踪产品的设计、开发、发布、运营等环节，确保项目按时完成和上线；6.数据分析与产品评估。定期对产品相关数据进行分析，善于从数据中发现问题，保持对数据的敏感度并指导行动。\\xa0任职要求：1.●数学、统计学、数量经济学相关专业，本科以上学历；2.●熟练掌握数据分析、自然语言处理（●NLP）基本方法论、2年以上互联网产品经验，有成功数据产品经验优先；3.●具备产品宏观视角，能独立负责产品设计和规划，良好的需求分析及挖掘能力；4.能精准地把握客户需求，善于进行前瞻性的用户需求判断、引导及控制，并落实项目需求；5.具备较强的逻辑思维能力、文案写作能力，善于合作协调沟通，思维敏捷；6.良好的团队合作精神，出色的跨部门协调和问题解决能力，能承担较大的压力。'",
 "'岗位职责：1、对物流大数据进行数据分析，建立安全数据模型；2、负责针对业务及用户部门的数据分析相关需求，进行需求解析和试验设计等；3、负责使用常用数据分析及数据挖掘算法来识别用户异常行为；\\xa0●4、负责模型算法开发与改进。●\\xa0●\\xa0任职要求：1、统计、运筹、数学、应用数学、物理、信息技术、计算机等相关专业本科及以上，三年以上工作经验；2、具有有丰富的大数据项目落地经验，能够将先进的机器学习算法应用到复杂的场景中去；3、对数据具有极强的敏感性，熟悉可视化技术，善于跟其他部门进行沟通；4、具有较强的团队合作和抗压能力，自主驱动，执行力强；5、至少熟悉一项以下方向：深度学习，强化学习，语音识别（●NLP），知识图谱。\\xa0●\\xa0\\xa0'",
 "'岗位职责：1.●负责分析游戏●活跃和商业化效果评估及问题定位，找到解决方案，持续监控改进效果;2.●负责输出游戏运营的分析报告，以及竞品分析和监控3.●负责游戏的数据化运营体系建设岗位要求：1.●至少1年的大数据库开发经验，包括数据报表开发、大数据计算优化等；2.●熟练使用hive●SQL；3.●有数据分析经验，游戏商业化、活跃等运营和分析经验的优先；4.●工作细致、责任心强，具有优秀的沟通能力和团队合作精神'",
 "'岗位职责：1.你需要负责各大产品数据统计、报表维护与监测，保证数据的业务针对性、逻辑性、准确性及可读性\\xa0；2.你需要结合业务目标梳理●KPI监测体系，进行重大产品数据监测与分析，为产品及运营优化提供建议；3.你需要主动发现和识别业务问题，规划分析思路，进行深入数据分析，为业务问题原因排查及解决方案提供数据支撑；4.你需要在深入理解业务基础上，结合业务目标，构建分析模型，为产品数据化运营提供系统化的解决方案；我们希望你：1.2年以上数据分析工作经验，1年以上第三方支付相关工作经验更佳；2.熟练使用●EXCEL、●PPT数据分析软件●R/●Python，●SQL，有较好的文字、数据、图表呈现能力；3.对数据敏感，具有缜密的逻辑思维能力及优秀的书面总结及口头表达能力；4.热爱数据分析，抗压能力强，能够独立完成数据分析报告，为领导提供决策支持。'",
 "'岗位职责:1.深入理解银行业务，通过数据提取、分析建模等方法，构建数据化产品，解决业务核心痛点；2.参与银行数据智能产品的研发工作，负责数据指标开发、业务数据探索等工作；3.利用●Hive●SQL查询工具、统计分析工具等，依托数据仓库进行数据的提取、整理、分析，发挥数据引领业务的作用；4.负责产品和模型的运营分析、跟踪评价、持续优化，通过数据全面反映产品的运营状态和问题。任职资格:\\xa01.熟悉数据仓库知识，熟悉●Hadoop数据库，掌握●Hive●SQL语言；2.具备扎实的数据提取、数据清洗能力；3.掌握常用的统计分析方法，具有一定的数据探索和分析能力；\\xa04.良好的沟通表达、团队协作能力；5.较高的责任心、担当能力；\\xa06.需要一年以上金融行业工作经验。7.此职位需要驻场办公'",
 "'岗位职责：1、负责●HR数据处理、各类基础数据报表/分析报表管理输出；2、负责制定并完善●HR数据维护标准，优化数据维护流程，监控数据质量；3、负责●HR数据统计分析、数据价值挖掘；4、严格基于流程操作，提供专业数据分析报告，保障服务质量，持续提升交付效率和服务体验。岗位要求：1、本科学历，1-3年工作经验；2、具有一定数据处理基础，对数据敏感，熟练使用excel等数据处理工具；3、有相关数据分析经验，保密意识强，对信息安全管理规范有一定了解。'",
 "'岗位说明：1、从事智慧城市、智能交通大数据相关的数据分析工作，涵盖数据采集、接入、清洗、分析、挖掘各个阶段；负责数据的逻辑分析和数据挖掘相关产品研发和维护，通过数据分析，支持决策2、协助公司大数据模块化平台的搭建与优化工作；3、负责海量数据采集、接入、清洗、分析、挖掘的代码实现；任职要求：1、全日制本科以上学历，数学、统计学相关专业，两年以上大数据分析工作经验；2、能熟练运用●Java、●Scala进行软件开发；3、掌握基本的算法和数据结构，掌握至少一种主流关系型数据库的使用和开发，掌握●SQL语言；4、熟练使用●Spark、●Hbase、●Flume、●Kafka、●ES等组件的原理，了解分布式系统开发，数据仓库技术者优先；5、强烈的责任心与求知欲，对技术有极大的热情，有良好的沟通能力。'",
 "'1、以帮助客户发现业务问题、发挥数据价值为导向，能深入理解客户业务需求与痛点，确定客户业务问题与目标，并给予有效的数据解决方案；2、从客户业务需求场景出发，指导/协助客户完成多维度指标体系的梳理和建设，并能完成数据采集、管理、分析与应用全流程工作；3、调动公司内部资源，完成神策分析系统在客户侧的部署及培训，帮助客户实现数据驱动业务的落地与迭代；4、以解决客户问题为目标，结合行业经验与方法论，提供咨询服务和专题分析报告；5、总结梳理垂直领域以及标准化实施流程/手册/文档，提升内部数据工作效率；6、梳理客户成功案例，形成行业指标均值，沉淀领域知识，形成知识库；7、与产品部门、技术部门配合不断提升数据产品的易用性，形成产品改进文档。岗位要求：1、五年及以上金融/互联网/咨询行业数据分析经验，本科及以上学历，数学及计算机相关专业优先；2、具备优秀的数据分析能力、以及业务感觉，有丰富的数据驱动业务的实战经验；3、熟练使用统计分析工具，逻辑严谨，能够独立分析和解决问题；4、具备优秀的沟通意识，良好的现场表达能力，能够独立负责和跟进项目；具备极强的学习能力和自我驱动意识，能够发现问题并解决问题；5、熟悉数据平台的整体搭建，熟练使用●SQL/●R/●Python/●Excel●等工具。'",
 "'岗位职责：1）负责海外金融业务数据挖掘、变量加工和分析等工作；2）与产品、技术的同事沟通协作，推动各类信用评估模型在具体业务层面和风控系统中的落地使用；3）参与数据源接入的相关数据需求整理和测试工作；4）跟踪、监测、维护及优化信用评估模型体系，确保模型的有效性；5）参加和跟进与风险管理相关的系统项目开发和测试工作。\\xa0任职要求：1）●1年以上风险建模、数据挖掘和数据分析相关经验；2）熟悉建模流程和原理，能够围绕业务和产品特征建模解决实际问题；3）熟练使用●R/python其中一种统计分析软件即可，熟练使用\\xa0●SQL。'",
 "'岗位职责：1、参与公司大数据平台建设，包括且不限于商品画像、采购备货、选型推荐等模型；2、负责销售/仓储采购数据分析，针对产品不同阶段构建科学的预测模型&策略规则；3、识别异常的销售、库存、采购●PO数据并迅速定位原因，提供数据分析以支撑业务决策；4、跟踪、监测、维护及优化商品运营体系，确保模型指标、数据的准确性；5、推动产品落地，并定期输出周期报告&专题报告。任职资格：1、计算机、数学相关专业，全日制统招本科及以上学历，两年以上商品方向相关数据分析工作经验。有丰富的商品运营、仓储管理、采购备货等经验优先；2、参于完整的数据采集、数据清洗、分析建模工作。具有快消、零售、物流行业工作经验优先；●3、熟悉多元线性回归、线性统计、时间序列模型；精通●Excel，熟练掌握●SQL、●Python等技能，有tableau/●Fine●BI或其他●BI软件使用经验优先；●4、有自驱力；善于总结，思维严谨，数据敏感度强；强责任心，良好的跨部门沟通能力和团队协作能力，对挑战性问题充满激情。薪酬福利：1、极具竞争力的薪酬，每年至少一次调薪机会和年终奖；2、月度评选优秀员工，优秀员工荣获丰厚的绩效奖金和优秀员工荣誉称号；3、透明的晋升通道、广阔的职业发展空间；4、入职后购买五险一金，享有年度体检；5、享有国家法定假期如年假、婚假、产假等；6、生育礼金、节日礼品和红包、生日礼品等；7、部门聚会、生日party、年会、打羽毛球、篮球、跑步等丰富多彩的文体活动。'",
 "'一、工作职责1.负责数据校对与报表更新更新，处理用户提出的新需求的前端报表开发，验证和测试；2.负责大数据平台（含tableau系统）运维与推广，包含用户培训：报表阅读培训，自助分析使用培训等；3.参与数据产品需求梳理，界面设计和产品测试验收；4.参与数据开发团队方案讨论和技术分享二、任职要求1.统招本科及以上学历，计算机、统计学、数学等相关专业毕业优先；2.2年以上数据开发工作经验；3.熟悉数据分析的方法，热爱数据分析工作；4.熟悉●SQL开发，掌握一种关系型数据库；5.熟悉任一主流数据分析工具，熟悉●Tableau，或●Smart●BI开发者优先；6.熟悉使用●Linux系统及shell脚本开发，熟悉●Python优先；7.学习能力强，工作态度积极主动，细致，对新技术好奇爱学习，主动性和责任心强，对数据敏感，逻辑性强，有良好的抗压能力。'",
 '0',
 '0',
 "'岗位职责：1、负责分布式网络爬虫系统设计，搭建系统框架，开发分布式网络爬虫系统；2、●完成数据采集爬取、解析提取、清洗入库等数据生产工作；3、研究网页特点和规律，对网页信息进行分类、抽取、数据清洗、存储结构等研发和优化工作；4、负责领域知识的定向爬取、深度提取和挖掘；5、对数据进行清洗、整理、去重及合并等工作；6、具有海量高并发和全网数据爬取经验；任职资格：1、本科学历，计算机相关专业，3年以上的软件开发经验；2、熟悉●Python语言，精通网络爬虫开发，有反屏蔽经验更佳；3、熟悉●XML、●HTML、●Java●Script、●JSON技术；4、熟悉●Mahout/●R/●MLLib中的一种算法框架；5、有一定的架构设计能力，有很强的分析、解决问题的能力；6、熟悉scrapy或selenium，熟悉mysql，mongodb等数据库，有数据库调优经验优先。'",
 "'职位描述：1、深入理解公司业务，完善数据监测、业务运营指标；2、服务公司各部门和项目，推动并配合数据分析类产品在各解决方案的落地和应用，为产品完善提供数据支持；3、负责产品的需求分析、指标定义、产品设计和产品迭代，输出高质量的●PRD文档。任职资格：1、统计、数学、计算机等相关专业本科及以上学历；2、二年以上数据分析或数据产品相关经验，参与过完整的数据分析项目；3、熟悉支付、餐饮、零售、金融等业务知识，有行业数据分析相关经验者优先；4、具有敏锐的数据价值理解和感知能力，具有优秀的产品设计能力，能快速落地抽象需求，并转化成产品形态。5、熟练使用相关数据分析工具（如●Python/●R、●SQL、统计分析●SPSS/●SAS、●BI工具●Tableau/●Power●BI）等，了解大数据平台架构；6、熟悉平台类产品规划方法，熟练使用●AXURE、●PPT、●WORD、●VISIO等产品策划工具。'",
 "'岗位职责：1.参与联合建模、特征变量系列产品和项目规划；2.结合数据挖掘，机器学习，深度学习算法，能独立构建在不同场景下的分类和预测模型；3.针对模型提出行之有效的优化方案；4.关注数据挖掘，机器学习行业动态，并思考与自有数据的结合点。任职要求：1.本科及以上学历，统计学、应用数学、计算机、金融等相关专业，3-5年工作经验；2.精通使用●Python/●R语言/●SPSS/●SAS中任一种进行数据分析与建模工作；3.熟悉各类模型分类与回归算法，如●LR、决策树、聚类算法、●SVM、●GBDT等，理解常用算法的来龙去脉及调优方式；熟悉建模过程中的数据处理方法，能基于业务数据独立完成模型构建及调优工作，并能协助团队成员解决问题；4.熟悉金融风控业务，具有良好的数据敏感度、逻辑能力与创新能力，能够提出自己对业务数据的思考与理解；5.能承受一定工作压力，有信用或欺诈风险建模经验，有海量数据挖掘的项目经验者优先，熟悉●Hadoop或者●Spark架构者优先；'",
 "'职责描述：1、跟踪信贷审批政策与规则的执行效果，出具分析图表及报告，给出优化方案；2、监测贷前贷中贷后各关键环节风险指标，对风险预警异常情况进行深入分析，并提供相应的解决方案及政策建议；3、针对各逾期状态、风险敏感性客群进行深度挖掘分析；4、配合公司业务开展，给出风险评估报告并给出资产分配的优化建议。任职要求：1、本科及以上学历（985、211优先），应用数学、统计学、计算机科学、运筹学等相关专业；2、2年及以上消金行业风险数据分析相关工作经验，对贷前、贷中、贷后其中一个或多个环节的业务数据有较高的敏感度；3、熟练运用●SQL、●Python，熟悉●Java；4、熟悉阿里云系列数据存储产品为加分项；5、有较强的逻辑分析、图表展示能力，能条理清晰地运用数据支撑相关的分析结论。'",
 "'岗位职责:岗位职责：1、负责游戏中心与小游戏的埋点设计、数据接入、数据仓库建设、报表体系建设、个性化标签建设等基础建设2、负责业务的关键指标波动监控、日常波动分析、专项分析等3、通过数据分析，发现业务的新方向与机会点，给运营提供可落地的建议岗位要求：1、本科以上学历，三年及以上大数据相关工作经验；2、熟练使用sql，hive等技术，了解hadoop工作原理；3、熟练使用linux操作系统，精通shell/python编程；4、熟练掌握数据分析工具（●R、python、excel至少一种），能够独立产出数据报告，熟悉数据采集到报表展示的全流程；5、有一定的业务数据敏感度，较强的数据分析能力、逻辑思维能力，以及分析问题、解决问题的能力；6、较强的沟通能力和团队合作能力，勇于承担工作压力。'",
 "'工作职责：1、优化广告投放的日报、周报、月报模板，提升报表自动化程度；2、挖掘投放有价值的线索，如人群/渠道/素材相关策略，建立测试机制，沉淀投放方法论；3、为公司业务竞标及复盘做数据支持，产出基于数据的洞察分析；4、协助后台搭建，提升部门取数效率。任职资格：1、本科及以上学历，1年以上工作经验，主动学习能力强，热爱电商/广告或有相关从业经验者优先；2、熟练掌握●Excel数据处理及可视化，熟悉●SQL/●Tableau/●Python优先；3、有用户行为分析/行业分析/竞品分析经验优先；4、性格积极开朗乐于交流，能承受一定工作压力。'",
 "'工作职责：1、深入理解并梳理公司各业务条线业绩指标，与业务部门一起对各节点的业务指标从数据层面进行分解、确认，并参与公司级业绩数据报表及业务数据追踪机制的建立；2、按职责负责公司业务条线业绩追踪报表的开发、准确产出及日常维护；3、利用专业的数据分析技术，与各业务部门合作，执行业务策略分析项目，提供先进的数据分析支持，保证分析的效率及项目目标的达成；4、从公司各业务数据源获取、清洗、整合客户和营销活动数据并将之融入到决策平台数据库的开发、管理和应用中，为公司级决策平台建设提供必要的支持；5、持续学习数据挖掘相关理论、工具及技术知识，保证技术水平的稳定维持及提升。\\xa0任职要求：1、本科及以上学历，统计学、数学、信息管理、计算机、市场营销专业优先；2、至少1年以上数据分析经验（或优秀的相关专业硕士以上应届毕业生），熟练使用数据分析语言或工具，如●SQL，●SAS，●Python，●R等。熟悉数据库营销或客户生命周期管理概念，有营销业绩追踪报表开发经验者优先；3、熟悉数据清理与整合、数据库模型、数据库的规划和管理（如●SQLServer、●Oracle等）。了解数据挖掘和多元统计分析原理，如逻辑回归、决策树、聚类分析、时间序列分析等优先；4、熟练使用●MSWORD、●EXCEL、●PPT、●OUTLOOK，英语良好，敏锐的分析能力，良好的沟通能力，积极主动，执行力强，能够承受较大工作压力。'",
 "'工作职责-●根据业务需求构建分析框架，将数据分析和算法转化为有价值的产品工具，帮助公司在商品/顾客/门店方向做出科学的决策-●参与用户成长路径的搭建，对顾客产品偏好及最佳触达时间点进行度量，从数据分析角度为运营提出建议-●深入理解全渠道零售业务，通过数据挖掘和机器学习技术，运用于客户行为分析及业务需求任职要求-●4年以上数据分析工作经验-●善于从商业角度发现和解决问题，拥有较强的数据敏感度和逻辑思维，能在业务中找出机会点-●熟悉数据库,●ETL,数据仓库，●BI系统；精通●SQL-●熟悉常用统计方法和机器学习模型，有推荐算法搭建与部署经验优先-●熟练使用数据可视化工具（●Tableau，●Congos，●Power●BI等），熟练使用数据●ETL工具（●Alteryx，●Datastage，●Kettle等），善于将数据分析结果转化为逻辑清晰、便捷使用的数据产品-●具有产品思维，能通过单一项目完成多业务需求-●有零售行业经验者优先'",
 "'1、结合京东数坊、京东九数、京东商智等平台的数据和工具，进行消费者洞察、行业洞察、细分市场洞察、竞品分析等，并构建消费者画像，输出策略洞察报告以及精准人群挖掘的策略建议；2、参与设计和构建消费者长期价值分析模型，为消费者资产增长以及分层运营提供策略性建议；3、结合应用需求，设计数据产品功能，撰写●PRD文档，并进行数据产品的项目管理；4、参与数据挖掘工作，负责部分数据建模和精准人群包的输出等工作。1、本科及以上学历，数学、数据挖掘、统计学、广告，市场营销等相关专业优先；2、3年以上数据分析、挖掘等相关工作经验，能独立完成数据清洗、数据分析、数据可视化的全过程，有独立制作过敏捷●BI或仪表盘的更佳；熟悉数据挖掘模型的特征工程、模型调参、模型评估、落地优化的全过程，熟悉随机森林、●GBDT、逻辑回归、●SVM等分类模型的更佳；3、熟练掌握任一种分析工具，例●Python、●SPSS、●SAS、●R语言、●Mat●Lab；熟练使用●PPT，能够根据策略洞察结果，独立撰写完整的营销策略方案；4、熟悉电商业务，对数字广告有一定了解的更佳。佛山\\xa0-\\xa0顺德区\\xa0-●北滘镇美的全球创新中心'",
 "'1.负责对接需求部门的可视化需求，参与报表和看板的原型设计与评审。结合对公司业务模型以及●BI平台的了解，提出报表和可视化看板的解决方案；2.负责需求业务底表的确认，参与数据统计底表的评审；3.负责业务报表和可视化看板的开发和运维；4.参与数据策略的制定，并推进应用落地，为线上线下零售业务赋能。1.本科及以上学历，计算机、统计、数学、信息等相关专业，3年以上相关工作经验优先；2.具备较强的数据分析能力和数据敏感性，有电商行业数据相关工作经验者优先；3.熟练掌握永洪●BI、●Fine●BI、●Tableau、●Power●BI等工具，有●BI开发项目实战经验者优先。熟悉数据库设计，了解存储过程和函数，有数据●ETL经验者更佳；4.有较强的学习能力、沟通理解能力和独自解决问题能力，有一定的创新意识；5.了解报表技术发展趋势，能够主动学习和掌握新的报表技术工具。广东省佛山市顺德区北滘镇美的全球创新中心'",
 "'岗位职责1.参与公司●BI建设，管理数据可视化自助平台，为各团队提供相关数据服务2.深入理解宏观及核心业务逻辑，搭建并持续优化业务数据分析体系，驱动业务运营和增长3.数据日常监控，协助产品、市场、销售等部门进行数据异常和波动诊断、快速定位问题、推动解决方案的落地4.管理建设公司报表体系，定期输出数据分析报告，为业务决策、产品方向、运营策略的调整提供数据支撑岗位要求：1.本科以上学历，数学/经济/统计学/物理/计算机等相关专业优先；2.在商业分析/用户增长/产品优化等领域有独立见解，精通●Excel，熟练运用●SQL、python，有●BI软件（●Tableau、●Power●BI等）、第三方数据分析工具（神策、●Growing●IO、●Google●Analytics等）使用经验更佳；3.有知名●Saa●S类产品/大型互联网企业运营分析、商业分析或数据产品岗经历优先；4.良好的逻辑分析能力和系统性思维能力，良好的跨团队的沟通能力，良好的数据呈现能力和观点陈述能力。'",
 "'【岗位职责】1.通过对用户行为进行数据挖掘和量化分析，为运营和产品提供深度洞见\\xa02.建设业务分析模型，不断评估和优化业务分析模型3.与产品,研发,运营密切合作,通过数据来发现业务的增长点4.支持业务上各种统计分析需求，快速跟进新业务的报表需求5.设计维护核心报表，帮助管理层和运营人员了解核心业务情况\\xa0【岗位要求】1.数学、统计学、计算机相关专业优先，对数据分析有强烈兴趣者优先\\xa02.科技公司一线产品或运营业务优化经验者优先3.熟悉●SQL和至少一种解释分析语言(●Python/●PHP等)4.有用数据方法对业务(产品,运营,策略等)产生过实质影响的经验5.熟悉常规数据分析方法，对自身的数据分析经历有系统沉淀，学习能力强，主动积极性高6.能快速理解业务，发掘业务细节和数据之间的联系，有良好的沟通能力，耐压力，和强大的推动力7.需具备2年及以上相关工作经验\\xa0\\xa0【加分项】1.熟悉flume,hadoop,spark,hive等大数据基础组件的部署优化2.熟悉●Davinci,●Grafana等大数据可视化组件3.熟悉python编程,尤其是熟悉●Pandas库'",
 "'职位描述：1、制定●FOF投资策略和资产配置计划，包括海外资产配置计划2、研究、开发量化投资策略，分析策略运行情况，并对量化模型进行优化和改进3、研究市场数据，发掘市场运行规律，提出和验证策略任职要求：1、全日制硕士及以上学历，金融与工科复合经验优先；2、具有2年以上资产管理行业从事基金投资相关工作经验；3、具有较强的资产配置模型和定量分析能力，精通投资组合分析、风险管理和优化方法；4、熟练应用python，my●SQL，能够独立完成量化投资策略的建模、开发、应用；5、具有强烈的责任感和良好的团队合作精神。'",
 '0',
 "'岗位职责：1.●GA（●Google●Analytics）设置及维护：设置●GA的各项配置，优化数据跟踪；2.●数据监测：对整体站内和站外的数据发生变化做实时监控，及时反馈异常波动；3.●数据分析：能通过分析用户行为、生命周期、留存转化等，给出提升产品运营效率和用户体验的建议；4.●活动分析:●针对各种营销活动，分析用户特征、需求、购买行为、活跃度等，提供数据可视化内容支持；5.●定期输出数据分析报告：负责各项日常运营数据的采集、统计与分析工作，优化网站运营工作；6.●跨部门协作：基于数据分析，与市场推广、运营、产品以及其他主要相关方开展跨职能协作。任职资格：1.数据分析能力强，熟悉及了解●GA（●Google●Analytics），有相关工作经验者优先2.●热爱互联网网络营销，熟悉各类网络营销3.●英文读写能力过关，英语六级及以上4.●有网站运营、●SEO、●SEM等工作经验者优先5.●做事认真负责，责任心强，有较强的沟通能力和团队协作能力'",
 "'岗位名称：资深数据分析师岗位职责：1、负责游戏平台数据抓取采集整理，包含市场分析数据，产品属性分析，竞争对手分析，属性分析、review分析等，确保数据以及数据挖掘需求真实可靠；2、针对产品进行用户行场景进行分析和挖掘，明确人群、环境、使用方式、用户付费挖掘、用户喜好等占比；3、根据已划分的产品线，进行产品池的整理，根据数据分析结果划分产品开发优先级，明确重点产品4、定期分析竞争对手游戏产品和新上竞品，提交竞品同比我们产品的优势、对手策略以及趋势分析；5、建立数据分析体系及各项报表，对数据进行项进行说明以及支撑依据，对数据进行建模定期输出数据报表，并能指出问题所在以及解决方向。6、协助产品进行数据报表规范化，搭建数据分析后台，并且推行内部数据分析的优化。任职资格：1、大学本科以上，985，211统招毕业生优先；理工科类专业，设计类（非艺术方向）、数学类、物理类、计算机类专业优先；2、3年以上相关工作经验，游戏相关相关行业公司的产品经理岗位经验、运营分析岗位经验、产品体验师岗位经验等优先；3、英语六级以上，办公软件精通，爬虫工具，数据分析工具；4、数理逻辑思维能力，发散思维能力，沟通能力；严谨，细心，理性，喜钻研，专注。'",
 "'岗位职责：1、负责产品的市场机会挖掘、竞争格局分析，产出基于数据的商业洞察，为业务决策提供关键输入；2、基于●BI平台数据，洞察生意问题和机会点，建立产品、业务的量化分析模型，追踪产品成长周期、市场趋势、价格策略等，为业务方提供决策支持；3、跟进业务方的日常运营数据，提供相应的业务场景的评估分析及特定问题的专项研究。任职要求：1、3年或以上数据分析相关工作经历，对业务逻辑和数据变化敏感，具备优秀逻辑分析能力和系统性思考分析问题能力。理工科背景，统计、数学、信息技术、金融工程等相关专业优先；2、对商业、市场和消费者有一定认知和理解。快消品、消费类电子行业，数据科学、数据分析、经营分析或数据化运营工作经验者优先；3、具备基础数据分析技能，熟练应用●SQL、●Excel、●Tableau、●Python/●R等数据分析工具。'",
 '0',
 "'岗位职责：\\xa01、负责公司核心业务数据的日常监控和分析，维护报表平台的数据输出；2、配合业务方，提供数据支持，输出专题分析及相关运营策略，推动业务转化效率提升；3、负责针对全国商户数据和用户数据进行深入统计分析，挖掘用户价值及业务增长点；4、从核心业务出发，负责宏观行业及市场竞争分析，为业务的整体策略提供支持和建议。任职要求：1、数学、统计学、计算机相关专业优先,3年以上工作经验；2、熟练掌握●SQL语言，熟练掌握hive、spark、python、●R语言、●SPSS、●SAS中的一种或者多种；\\xa03、逻辑清晰，数据敏感度高，沟通能力强，业务理解能力强，具有吃苦耐劳敬业精神和团队精神；4、有●B端数据分析相关经验优先。'",
 "'岗位要求：\\xa01、工作细心，逻辑推理能力强。2、具有团队合作精神，能承受一定的工作压力●。3、精通办公软件，熟悉常用分析方法，有数据分析经验。4、熟悉●Facebook、●Instagram、●Twitter、●You●Tube等社交媒体渠道。5、英语六级以上。岗位职责：1、收集并分析市场信息，分析竞争对手掌握市场动向，寻找机会。2、行业动向分析，产品发展方向分析。3、专题信息搜集与分析。4、●精通办公软件，熟悉常用分析方法，有市场数据分析经验优先考虑。'",
 "'岗位职责：1、商业嗅觉灵敏，善于沟通，能够深入了解业务，挖掘业务问题和痛点，通过商业分析为公司运营决策、产品规划、运营规划提供数据支持；2、针对运营问题进行深度诊断，发现业务运营问题与机会点，辅助制定运营策略，提升运营效率；3、整合运营数据分析与应用需求，设计运营相关数据产品；4、参与数据分析体系的完善，撰写数据分析报告为公司运营、管理等提供数据支持；5、负责公司基础数据的架构与维护。任职资格：1、大学本科及以上学历，●数学、统计、经济等相关专业优先，3-5年数据分析相关从业经验；2、具有快速学习和思考能力，优秀的商业分析能力、敏锐的数据洞察力、结构化思维能力；3、精通excel操作（常规函数、数据透视表、模型搭建），熟练使用●SAS、●Python、●SPSS等进行数据分析工作；熟练使用●SQL；4、对数字敏感，做事细心、有耐心，责任心强，具备沟通协调和闭环管理、推进能力。'",
 "'岗位职责：\\xa01、负责公司网店业务数据的日常监控和分析，维护报表平台的数据输出；2、负责分析网店流量的来源及转化漏斗，输出专题分析及运营/推广策略；3、与产品/运营/研发配合，推动网店相关产品迭代优化，输出迭代分析报告；4、负责针对海量用户数据进行深入统计分析，挖掘用户属性，推动转化效率提升。任职要求：1、数学、统计学、计算机相关专业优先,3年以上产品运营数据分析工作经验；2、熟练掌握●SQL语言，熟练掌握hive、spark、python、●R语言、●SPSS、●SAS中的一种或者多种；\\xa03、逻辑清晰，数据敏感度高，沟通能力强，业务理解能力强，具有吃苦耐劳敬业精神和团队精神。'",
 "'●HR告白：*●上班不用打卡*●正式员工享受7天带薪年假（创业友军听了会流泪）*●入职即购买五险一金，丰厚年终奖（0~3个月）*●企业文化简单，不灌鸡汤不洗脑，推崇独立思考、自我驱动【岗位职责】1.●推动公司数据化运营体系的建立，推动数据标签化，并对标签数据进行管理、分析及使用;2.●完善用户画像，针对用户营销做好数据支持;3.●负责日常数据收集、统计、分析，建立和优化指标体系，监控数据的波动和异常;4.●新业务数据建模，支持业务进行数据分析，支持管理团队进行业务决策；【职位要求】1.●有良好的统计知识，具备电商、在线教育、金融行业从业经验;2.●擅长●SQL，了解索引机制，有●SQL优化能力；3.●至少会一门脚本语言例如●Python等；4.●有多种数据库经验(关系型、非关系型，比如mysql、mongo●DB)、大数据经验(adb、es、tidb等)加分项：-参与过大型项目或系统建设的经验-具备流量分析系统、管理决策系统、经营分析系统建设经验者；【●Why●Join●us】1、我们选了一个不错的赛道教育是最抗经济周期的行业。2019年已经过去，诸多曾经的明星创业公司跌落“神坛”，资本寒冬悄悄来临，然而在线教育仍是今年创投圈的热门话题和风口。【叮咚课堂】运用创新的产品模式和科技手段，把昂贵的在线直播课堂变成平价课堂，通过互联网●AI技术，让平民家庭的小朋友也可以随时随地低成本和北美外教老师互动学英语。2、我们有一群教育业内领先的创投人支持我们在2018年3月起步半年时间，迄今已获险峰长青、创世伙伴、襄禾资本三家知名●VC的大力支持，短短两年时间已有4家一线基金投资和跟投。3、我们的文化：一个务实的团队团队核心成员均来自腾讯阿里等一线知名互联网及教育机构，创始人有丰富的教育+互联网项目实战经验，我们的产品、技术、学术、运营未来一年还在招兵买马，无论你是单兵作战能力强还是组队能力强，我们虚位以待。4、我们愿为你提供：丰厚的回报不低于该岗位的市场薪资报酬入职即购买社保五险一金、每年都有团建旅游、优美的办公环境地处南山高新园地铁站，交通便利好打车灵活调薪，如果你是快速成长型选手，我们的薪资涨幅也会快速跟上你的成长速度'",
 "'岗位职责：1、基于海量的玩家行为数据，对于玩家进行画像；2、通过爬虫工具，收集网站玩家数据，进行分类画像；3、通过算法，预估一段时间内游戏的●DAU和收入走势；4、通过玩家行为研究，改善游戏体验和游戏生态。任职资格：1、计算机、统计学、数学相关专业，3年以上实际数据挖掘建模项目经验；2、熟悉数据挖掘建模过程及主流算法及相应数据分析与处理技术；3、掌握数据挖掘理论及较为扎实的统计理论知识，熟练使用至少一种专业统计分析软件，如●SPSS、●R等，熟悉hadoop；4、具有独立分析和解决实际业务问题的能力；5、热爱游戏，有高度的工作责任心、良好的团队意识及独立工作能力。'",
 "'岗位职责：1.●\\xa0建立并完善数据体系，制定整体业务数据获取规范、流程、标准，并监督执行、优化；2.●根据业务部门运营需求，梳理标准化数据指标，并建立、维护自动化报表体系，监控并反馈业务运营状况；3、深入理解业务及产品，跨部门协调和沟通，响应临时性数据分析需求；4、分析数据成因及业务运营关联性，发现问题并为企业决策提供支持。岗位要求：1、本科以上学历，统计、数学、计算机信息工程等专业优先；2、三年以上互联网行业工作经验，五年以上数据分析经验；3、具有扎实的统计学、数据分析、数据挖掘基础，熟悉数据分析、数据挖掘的手段；4、对数据敏感，归纳、分析能力强，做事严谨细致；5、有数据平台搭建等相关经验；6、精通office系列软件，熟练access、mysql等数据库软件，熟悉●SPSS、●Python、●MATLAB等软件'",
 "'工作职责：1、配合数据研发搭建数据可视化平台（●BI）；●2、制定营销活动核心指标和复盘数据分析；3、通过用户分层数据的深入分析，建立用户生命周期管理体系，发现用户增长关键因素，帮助市场团队寻找提升用户促活方案。4、营销活动和工具促活效果分析，优化营销工具使用策略。任职要求：1、4年以上工作经验（优秀者可放低经验要求），本科及以上学历，计算机、统计学、数学等相关专业优先；●2、较强的统计学基础，优秀的数据处理能力，敏锐的数据洞察力，良好的业务理解力；●3、熟练掌握数据分析方法与模型；●4、能独立完成从数据提取到模型构建，部门沟通到输出分析报告，提出并解决具体的业务问题。'",
 "'岗位职责：1、负责新游数据相关工作，通过数据分析和数据挖掘，找出游戏的问题，调优建议和方向；2、负责游戏数据分析体系搭建，包括●LOG埋点、日志上报、分析系统的规划等；3、负责游戏目标用户分析、数据预估、用户调研等，为运营计划及市场方案提供依据；4、负责游戏上线及版本更新时的效果分析，包括：留存及流失分析、付费分析、功能玩法分析、用户满意度调研等；5、通过数据监控游戏中的各种动向与产品健康度，及时发现不良运营情况。岗位要求1、3年以上游戏数据分析经验，对用户建模、用户调研等有一定的了解；2、对数据敏感，有较强的逻辑思维和数据分析能力，并有大数据对比及处理经验；3、熟悉游戏产品运营工作，了解用户行为模式和心理；4、较强的总结和报告撰写能力，熟练使用excel、word、ppt等工作软件；5、良好的沟通能力和合作意识，抗压能力强；6、了解各类游戏产品的表现情况，对沙盒游戏产品有一定了解。'",
 '0',
 "'1、有3年以上数据治理、数据分析经验；\\xa02、很好的业务和产品理解能力；\\xa03、熟练使用●SQL/●Shell/●Python等语言；\\xa04、熟练使用●Hive/●Spark/●Kafka/●Flink等大数据组件；\\xa05、熟悉机器学习相关算法优先。'",
 "'岗位职责1、搭建并完善网游数据分析框架体系，完成从数据埋点、数据接入、数据分析、数据可视化、专项数据分析报告输出等数据分析工作，全面推进各项目的数据化运营；2、建立游戏关键指标的日常监控机制，对游戏数据波动进行深度挖掘，推动游戏内容及运营策略调优；3、综合使用数据分析、用户研究等多种手段，对游戏版本、运营活动等内容进行专项研究，支撑研发决策；4、基于数据分析结果推动问题解决或游戏优化，并持续关注后续数据表现及玩家反馈；5、建立网游数据分析模型和预测模型，推演游戏生命周期发展趋势；6、负责接入并持续维护数据分析相关的管理工具。任职要求1、本科及以上学历，熟悉并热爱互联网游戏行业，有丰富数据分析或商业分析相关工作经验；2、在数学、统计学邻域有扎实的理论基础和应用能力，熟悉使用●SQL查询语句、●R等数据分析工具；\\xa02、对数据敏感，有较强的数据分析能力，能够在海量数据中及时发现问题；3、细心、敏锐，理性思考，抗压能力强，性格积极外向，有团队合作意识；4、工作有计划性，善于时间管理，工作效率高。'",
 "'1.\\xa0\\xa0\\xa0\\xa0根据客户的业务场景，帮助客户梳理业务需求，搭建业务指标体系，完成数据监控及分析2.\\xa0\\xa0\\xa0\\xa0通过对数据的敏锐洞察以及定性和定量分析，发现问题和机会点3.\\xa0\\xa0\\xa0\\xa0能够根据实际业务，完成深入的专项数据分析，提出策略，形成数据分析报告4.\\xa0\\xa0\\xa0\\xa0围绕用户行为展开研究，与客户一起构建增长实验，通过实验和分析总结，实现数据驱动5.\\xa0\\xa0\\xa0\\xa0基于用户行为、标签和生命周期，通过统计分析、建模等方式，构建不同场景下的增长模型6.\\xa0\\xa0\\xa0\\xa0与内部客户服务团队、产品/研发团队以及其他主要相关方开展跨职能协作，助力相关产品/服务取得成功1.\\xa0\\xa0\\xa0\\xa0统计学、数学、计算机等相关专业学历，在用户增长/产品优化/商业分析上有3年及以上相关经验者优先2.\\xa0\\xa0\\xa0\\xa0具有较强的业务场景理解和分析能力3.\\xa0\\xa0\\xa0\\xa0统计基础扎实，熟练使用●SQL、●Excel，具有独立建模能力，掌握●SPSS、●R等其中一款统计分析工具4.\\xa0\\xa0\\xa0\\xa0具备较强的表达和主动沟通能力，擅长与内部及外部合作团队交流沟通5.\\xa0\\xa0\\xa0\\xa0具备良好的逻辑思维和组织能力，能够独立分析和解决问题6.\\xa0●\\xa0\\xa0有互联网+相关的数据平台项目经验或者数据仓库项目经验者优先；'",
 "'岗位职责:1、深入了解公司线上线下业务，门店运营流程，供应链计划库存等，提供数据决策支持；2、参与数据体系的建设，探索分析模型，实现数据为需求部门赋能，提高业务决策质量；3、通过数据分析和数据模型等数据分析能力，针对公司运营决策、产品方向、销售策略，提供或辅助优化运营策略或产品方案建议；4、收集产品团队和其他业务团队数据洞察需求，抽取需求共性，基于●BI可视化平台推行数据监测预警，数据可视化等。任职资格:1、计算机，统计学，数学，经济学等相关专业，本科及以上学历；2、对互联网行业有比较深入的了解，有互联网，餐饮行业经验者优先；3、具备较强的团队协作能力和沟通能力，逻辑思维活跃，学习能力强；4、具备较强的分析问题和解决问题的能力，有强烈的责任心；5、精通使用●SQL等数据查询语言，熟练使用python，●R语言更佳。'",
 "'岗位职责：1、基于业务理解，构建完善的业务监测指标体系，有效披露业务进展与风险，保障业务策略落地；’●2、通过深入的专题分析，定位业务问题，提出有价值的业务机会或方案，辅助业务决策；●3、独立承担复杂分析任务，擅于运用各类数据分析技术，实施深度数据诊断，并能沉淀关键分析思路，推动产品化进程；●4、积极进行跨部门沟通与协作，主动推动业务发展和体验改善。任职要求：1、五年及以上相关工作经历；●2、统计、数学、计算机本科及以上学历优先；●3、有丰富的数据分析挖掘经验，并能灵活运用于实践。有较强的逻辑思考能力，能独立撰写思路清晰的分析报告；●4、较强的商业观察能力和判断能力，能主动推动业务落地；●5、掌握●SQL等代码编写，熟悉常用算法模型，如聚类、关联和分类等，熟练操作●Excel、●R等工具。'",
 "'【岗位职责】1、将主策划的数值预期转化为可执行的数值公式和表格模型。2、设计游戏系统数值框架及数值操作流程。3、负责游戏中各系统实现的数值支持。4、负责游戏中经济系统的数值设计和平衡。5、进行游戏运营期的数据分析，数据挖掘，改善用户体验和付费情况。【任职资格】1、本科及以上学历，数学、统计学、信息计算科学等相关专业背景，有较强的数学逻辑和数学功底（优秀应届毕业生亦可）。2、对游戏平衡性及数值方面有较为深入的研究，熟悉玩家心理；有能力主导完成游戏系统、数值、关卡的设计。3、至少主导过一款以上成功上线的游戏产品的数值策划，熟悉游戏研发流程；参与过棋牌娱乐类游戏的数值策划工作的优先。4、熟练掌握mysql，熟悉●R、●SPSS或者●SAS等数据分析工具之一；熟练使用办公软件，精通数值建模与运算，可熟练使用●VBA者优先。\\xa05、热衷游戏制作，学习能力强，思维活跃、逻辑清晰。6、适应能力强、良好的沟通表达能力，对工作充满热情，富有责任心，良好的团队合作精神。\\xa0'",
 '0',
 '0',
 "'岗位职责:•●结合前沿的研究成果，对●QQ社交网络进行挖掘和研究，包括网络结构、群体/个体行为、信息传播等方面，为产品方向和建模落地提供指导；•●用机器学习、文本挖掘等技术，对结构化和非结构化数据进行挖掘，发现其潜在关系，指导●QQ大数据应用落地。岗位要求:•●数学、统计、计算机等相关专业硕士及以上学历，三年以上互联网数据挖掘和应用相关工作经验；•●对机器学习、数据挖掘、自然语言处理等有较好的理论基础或实践经验•●良好的团队合作精神，较强的沟通能力，很强的自我驱动力'",
 '0',
 "'职责描述：1、腾讯●Next学院是是腾讯课堂子品牌，是腾讯自研●IT课程的生产者，腾讯●Next学院旨在提供最实用、最前沿的技术干货给用户，成为在线教育的变革者，助力人人都是工程师的未来世界。2、无须全职，在不影响本职工作的前提下，能较自由地安排自己的工作时间，并能按照规定时间完成课程制作；没有工作地点限制3、讲课形式为直播课，直播课时间为每次1-2小时，无须露面，每月1次4、课酬丰富，是行内发声的好机会任职要求：1、主要负责课程内容的设计与制作2、python数据分析:从事数据分析挖掘相关工作，精通●Python，擅长●Python数据分析、爬虫等。3、●Excel+●SQL数据分析:丰富的数据分析经验，精通●EXCEL和●SQL。微软●Excel认证专家优先。加分项：腾讯内部工程师优先大厂背景开发经验优先'",
 "'岗位职责:1、负责分析挖掘用户行为，帮助产品、业务线发现、分析问题，并提供决策数据。2、负责通过专项数据分析对业务问题进行深入剖析，为业务产品、运营、●BA决策提快数据支持。3、参与产品、业务平台数据化运营，构建丰富多样的●BI报表，助力产品、业务不断优化。任职资格:1、从事数据分析领域1-3年工作年限，具备仓库模型、●ETL任务设计、开发经验。热悉●Linux系统常规shell处理命令.灵活运月●Shall做的文本处理和系统操作。2、烈悉●Python、●SQL语言，了解●Hadoop生态相关组件原理、并具备一定开发经验，包括●Hive、●HUE、●Presto等。3、有金融业务数据分析、数据仓库经验优先。4、良好的语言沟通与表达能力、较强的自我驱动能力。'",
 "'数据分析总监职位描述：工作职责:\\xa01、负责数据框架搭建：设计合理的数据指标体系，与数仓同事配合高效实现，为用户运营、商户运营、产品、市场等决策提供数据支持和建议；\\xa02、负责数据产品的设计落地：能根据业务特点，选择或设计数据产品，提高全公司的数据应用能力和效率；3、专题分析：贴近业务，对业务问题进行深入分析，及时发现运营及产品问题，为业务运营决策、产品策略、产品方案提供数据支持，给出分析报告；4、对数据准确性负责，体系化地解决数据准确性问题；5、负责团队建设及人员培养。\\xa0任职资格:\\xa01、本科学历需要有5年以上相关经验，其中2年以上管理团队经验；2、熟练使用sql，熟悉各种数据技术平台，例如：hadoop、mysql等，能高效的与数据技术团队进行沟通；\\xa03、熟练独立编写数据分析报告，及时发现和分析其中隐含的变化和问题；4、了解互联网常用的数据分析和数据运营的方法和思路；有客户数据模型建立、数据化运营经验优先考虑；同时具有●C端和●B端数据分析经验的优先；有商业分析背景优先；有数仓设计、建设或数仓团队管理经验优先；5、良好的逻辑思维能力和数据洞察力，能够发现数据中有价值的信息。'",
 "'工作职责:1、全方位分析业务状况，判定业务发展趋势，为公司运营决策、产品方向、销售策略提供数据支持；2、监控业务数据，分析诊断业务运营中的问题，并推动业务问题的解决；3、进行用户行为分析，构建平台用户画像，搭建模型挖掘用户群体行为特征，提升业务指标和用户体验；4、聚焦业务问题，开展业务调研，独立编写商业数据分析报告，及时发现和分析其中隐含的变化和问题；5、基于数据使用场景，对数据进行梳理，推动底层数据建设，负责制定数据采集方法、数据使用规范、口径管理、原数据管理、数据清洗等工作；6、沉淀思路与框架，提炼数据产品需求，与产品、研发等团队，协作并推动数据产品落地，实现数据产品化。任职资格:1、全日制本科以上学历，要求统计、数学、●IT类、工程类等专业，一年以上互联网公司数据分析相关工作经验；2、具有较强的商业、数据和业务的敏感性，具备良好的逻辑分析能力，能够系统性的思考和分析问题，同时具有较好的分析总结能力和数据报告呈现能力；3、精通●SQL，具备熟练使用●Python进行数据分析的能力；4、有强烈的责任心、沟通、协调能力以及团队合作精神。5、熟悉●Hadoop生态，熟悉●Hive/●Presto，有海量数据的分析和挖掘经验者优先；6、熟悉●Tableau等●BI者优先；'",
 "'岗位职责：1、根据业务部门需求，充分利用现有数据资源，进行特征工程，构造有效合理的特征，建立合适的分析模型对业务问题进行分析；2、根据院校历史最低录取排名、招生人数等数据，构建特征、建立预测模型，计算院校录取概率；3、结合现有业务数据分析结果以及用户个性化需求，为用户推荐合适的内容；4、基于具体业务规则，文本相似度等分析方法，进行数据清洗工作。岗位要求：\\xa01、有数据分析/挖掘经验，有互联网行业分析经验优先；\\xa02、硕士学历，数学、统计、计算机专业优先；\\xa03、参加过完整的数据分析项目，能够独立开展研究项目和撰写分析报告；较强的数学、统计和计算机知识，熟悉特征工程，回归分析、概率分布等统计知识；4、熟悉●SQL，熟练掌握python，加分项：熟悉flask框架、docker、git，●Go语言；5、具备良好的沟通能力，能够进行跨团队的沟通协调。'",
 "'工作内容：1、负责公司运营数据的基本处理、维护及相关报表制作；2、撰写数据分析报告，能从数据分析角度为公司运营决策、产品方向等提供数据支持，并对运营效果进行评估分析工作；3、负责业务相关数据的统计分析，每日进行统计与监控；4、与各部门保持良好的沟通，了解其数据收集及分析要求；处理业务相关的临时需求，并对于常用重要需求转化成数据产品；5、上级领导安排的其他工作。任职资格：1、本科及以上学历，数学、统计专业优先；2、●2年以上互联网数据分析、运营分析等相关工作经验，熟悉数据分析方法和思路，具备独立完成分析报告的能力；3、具有良好的条理性和逻辑分析能力，除数据处理能力外；4、熟练掌握●My●SQL，精通●EXCEL，●PPT，能利用python进行数据分析和处理优先；5、对机器学习有一定的了解，熟悉常用的算法模型，有实际业务建模经验优先。6、有较强的学习能力和探索能力，能够积极主动发现问题；'",
 "'岗位职责：1、独立进行需求沟通、方案编制、数学模型开发；2、独立完成常见大数据分析建模工作，完成数据分析报告、汇报材料等文档编辑3、通过大数据挖掘用户群体行为和属性特征，监测分析产品运营数据等，并将数据推送至各业务部门，从数据角度支撑业务运营的决策，不断推动业务优化;4、负责定期为公司运营层面提出企业经营状况分析和前景预测报告；5、满足跨境电商对行业动态、经济、市场数据、仓储，物流，进出口的分析需求，并建立数据模型任职资格：1、统计学，数学等专业本科及以上学历，2~3年以上数据分析工作经验2、熟悉●Python至少一种编程语言3、具备良好的文档编辑能力，从事过跨境电商数据分析尤佳，熟悉跨境电商行业业务流程优先'",
 "'1.根据业务流程搭建数据分析体系，统一数据口径，并将分析体系在●BI平台上可视化；2.及时响应业务团队数据统计分析需求，输出可读性强的分析报告，结合数据趋势对公司产品的运营和决策给出专业的建议；3.根据应用产品的需求，设计数据算法方案，完成方案落地和复盘。1.良好的逻辑思维和批判性思维能力、良好的数据敏感度，能从海量数据提炼核心结果，●能够以客观的数据和严谨的分析过程，形成可靠的分析结论及建议；2.3年以上数据分析经验，熟练运用●SQL,●有互联网数据分析背景者或有使用●Hadoop、●Hive、●Spark处理数据经验者优先；3.熟悉目前市场主流的报表工具，如：●ORACLEBIEE，●Tableau，帆软●BI，永洪●BI、●Quick●BI、●BDP等；4.有高度的责任感，有上进心和主动性，独立主导和统筹过项目者优先。'",
 "'岗位职责：1.负责平台、垂类业务数据分析，定位平台及业务发展存在的问题，为业务线运营提供决策依据。2.深入理解业务，开展专项分析，通过对产品与用户行为数据进行分析和挖掘，输出专题分析报告，为产品和运营提供相应建议，并推动落地为线上实验，形成闭环。3.通过数据分析工作，优化分析模型，沉淀分析方法论，针对不同业务场景，形成可复用的数据分析框架。4.负责推动数据分析相关工具开发与规划，为业务分析工作做有力支撑。任职要求：1.本科以上学历，应用数学、统计或计算机专业方向等相关专业优先；2.至少有3年互联网数据分析/策略分析/用户分析工作经验，有●OTT行业经验优先；3.熟练使用●SQL，熟练使用●Excel、●SPSS等统计分析工具，对数据分析与处理有一定的经验；4.有较强的逻辑思考能力和结构化思维，做事严谨客观，对工作投入专注；团队合作意识强。'",
 "'职责描述：1、参与●BI相关大数据系统的需求对接、架构设计、开发工作；2、参与需求构建数据仓库及数据集汇总优化，协助后台技术团队完成●DW、●ETL功能实现及相关●IT项目；3、对●BI用户数据分析技术支持、●IT技术支持、用户培训和平台运营提供数据支持。4、针对业务部门提出的各维度数据问题，进行数据获取，整理和分析，得出合理的结论，并以报告的形式呈现；5、深入发掘运营业务中的异常数据，主动分析并提供结论，推进业务部门调整及优化；6、负责规划和实施中对各业务主题进行分析及优化；7、开发和维护系统平台与框架，发现和解决存在的技术问题，保证●BI系统的性能和稳定性；8、负责项目级（复杂）报表需求进度，参与实施、推进项目达成验收；9、支持业务部门对业务数据进行挖掘，找出核心问题，并提出关键举措，推动运营指标及业务能力的提升级。任职要求：1、本科及以上学历，计算机科学与技术、统计学、数学、数据科学、财务会计等本科及以上学历；2、3年及以上的●BI项目及数据分析工作经验；3、熟练使用至少一种关系型数据库（●Oracle,●Sql●Server,●My●Sql），有数据库优化经验，有非关系型数据库使用及●PL/●SQL等脚本编程经验优先；4、对数据敏感，具备优秀的分析技巧以及报告展示能力，了解主流报表工具或新兴前端报表工具；5、熟悉电商平台企业的●ERP中台、电商●EC\\\\●OMS、●WMS、财务金蝶等应用系统操作更佳；6、有管理和分析用户需求的经验，有平台商城各行业数据分析和数据洞察的技术支持经验优先；7、有较强的计划和组织协调能力，具备结果导向思维，工作踏实、对技术工作有热情。'",
 "'职位描述：负责游戏上线及版本更新时的效果分析，包括：留存及流失分析、付费分析、功能玩法分析、用户满意度调研等；任职要求：1、有电竞行业数据分析经验，分析能力强，逻辑清晰；\\xa02、通过数据分析，能够较准确预测英雄联盟、王者荣耀、反恐精英、刀塔2、守望先锋、炉石传说、星际争霸2等游戏赛事情况，准确率不低于70%；3、有较强的文案撰写能力，能够独立撰写游戏分析报告；'"]

In [4]:
create_stats_word_table('',info_List)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sl/q8x6_03132dfk7rktf00yh880000gn/T/jieba.cache
the first step need , list text and check it 1.9073486328125e-06
Loading model cost 0.787 seconds.
Prefix dict has been built succesfully.
the second step need , create feature dict 13.254734754562378
the third step need , create feature vector 13.255255937576294
The fourth step need ,get all words dict cosangle  25.08208990097046
{'count': 102, 'list text': 1.9073486328125e-06, 'statistical frequencyy': 13.254732847213745, 'create frequency vector': 0.0005211830139160156, 'cosl': 11.826833963394165}


,cos,cosangle,radian,words,sentences,good,joy,sorrow,anger,fear,evil,fright,entropy,density
0,0.074733,0.469561,0.001304,579,29,28,14,1,0,1,6,0,12.165552,0.041100
1,0.097594,0.613199,0.001703,410,17,16,4,0,0,0,3,0,14.687747,0.068957
2,0.059996,0.376968,0.001047,360,24,13,3,2,0,0,0,0,11.295976,0.068878
3,0.033105,0.208005,0.000578,119,8,4,1,0,0,0,1,0,7.511780,0.139107
4,0.044882,0.282000,0.000783,195,19,10,2,0,0,0,0,0,9.312102,0.104630
5,0.088049,0.553229,0.001537,333,12,14,1,0,0,0,0,0,14.949588,0.083053
6,0.000000,0.000000,0.000000,1,1,0,0,0,0,0,0,0,0.000000,NaN
7,0.075867,0.476683,0.001324,276,11,14,1,0,0,0,1,0,11.804192,0.083128
8,0.000000,0.000000,0.000000,1,1,0,0,0,0,0,0,0,0.000000,NaN
9,0.055875,0.351071,0.000975,181,10,11,1,0,0,0,0,0,9.759987,0.102737


In [5]:
file_path = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP数据集合/豆瓣电影数据集(2019.3)/豆瓣电影数据集(2019.3).xlsx"
file = pd.read_excel(file_path)
textdata = file.剧情简介.to_list()

In [12]:
textdata = [par for par in textdata if isinstance(par,(str,bytes))][:10]

In [14]:
create_stats_word_table('剧情简介',textdata)

the first step need , list text and check it 1.9073486328125e-06
the second step need , create feature dict 0.9400551319122314
the third step need , create feature vector 0.9405100345611572
The fourth step need ,get all words dict cosangle  1.866852045059204
{'count': 10, 'list text': 1.9073486328125e-06, 'statistical frequencyy': 0.9400532245635986, 'create frequency vector': 0.00045490264892578125, 'cosl': 0.9263420104980469}


,剧情简介cos,剧情简介cosangle,剧情简介radian,剧情简介words,剧情简介sentences,剧情简介good,剧情简介joy,剧情简介sorrow,剧情简介anger,剧情简介fear,剧情简介evil,剧情简介fright,剧情简介entropy,剧情简介density
0,0.152078,0.955536,0.002654,191,7,10,3,1,1,2,11,1,7.697798,0.082772
1,0.137134,0.861639,0.002393,104,6,1,0,1,1,1,3,1,7.897297,0.138549
2,0.044832,0.281690,0.000782,55,2,2,0,0,0,0,2,0,4.800376,0.200016
3,0.261961,1.645952,0.004572,166,5,1,0,0,0,1,2,0,9.235852,0.093291
4,0.051865,0.325876,0.000905,72,3,3,0,0,0,0,2,0,6.384936,0.168025
5,0.045711,0.287213,0.000798,80,4,1,0,0,0,0,2,1,5.392317,0.128389
6,0.123069,0.773266,0.002148,185,5,1,2,1,1,2,3,0,7.444643,0.077548
7,0.033404,0.209886,0.000583,66,4,0,0,0,0,0,2,1,5.208406,0.186014
8,0.209217,1.314552,0.003652,170,6,4,0,1,0,1,6,0,8.370045,0.093000
9,0.159990,1.005246,0.002792,191,6,6,0,2,0,0,3,0,8.494221,0.080897


In [52]:
from itertools import permutations
import re

def create_char_map(str_range = 'lowercase',chinese_path = False):
    iter_range = lambda char_range : map(lambda x : chr(x),char_range)
    func_dict = {'lowercase':iter_range(range(97,122))
                ,'uppercase':iter_range(range(65,90))
                ,'numbers':iter_range(range(48,57))}        
    if chinese_path:
        func_dict.update({'chinese':(char for char in open(chinese_path).readlines())})
        return func_dict["chinese"]
    else:
        return func_dict[str_range]

def collate_char_iterator(itertools_perm):
    return map(lambda x:"".join(x),itertools_perm)

def chargen(language="lowercase",n=1):
    return collate_char_iterator(permutations(create_char_map('lowercase'),n))

def add_char(input_char,language="lowercase",n=2,forward=True):
    return ("{}{}".format(char,input_char) 
        if forward==True else "{}{}".format(input_char,char) 
        for char in chargen(language=language,n=n))

def replace_char(input_char,language="lowercase",n=2):
    m = len(input_char)
    S = chargen(language=language,n=n)
    for create_str in S:
        for i in range(m):
            result = yield input_char.replace(input_char[i:i+n],create_str)

symbles=''':,"{[}](>)</\n。●  ，、的 啊 好 和
并 与 及 对 错 你 我 我们 她 他 它：: ; ；《 》
1 2 3 4 5 6 7 8 9 0  ‘ “ ” ’ + - * / ` ~ 
\( \ [ \ { \ } ] ) （ ）【 \xa0 】理想 愿景
工 不管 只要 一员 大家庭 当成 作 帅哥 美女 年轻
佛系
'''
def delete_element(strings,symbles=symbles):
    srcrep = {i:'' for i in symbles }
    rep = dict((re.escape(k), v) for k, v in srcrep.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], strings)

def delete_char(input_char,language="lowercase",n=2):
    return (delete_element(input_char,"".join(chars)) for chars in permutations(input_char,n))

def translation_str(input_char,language="lowercase",n=2):
    del_ = delete_char(input_char,language=language,n=n)
    replace_ = replace_char(input_char,language=language,n=n)
    add_forward = add_char(input_char,language=language,n=n,forward=True)
    add_backward = add_char(input_char,language=language,n=n,forward=False)
    return tuple(list(del_)+list(replace_)+list(add_forward)+list(add_backward))

def translation_n(input_char,language="lowercase",n=2):
    result = []
    for i in range(1,n+1):
        result += list(translation_str(input_char,language=language,n=i))
    return result

In [54]:
%time len(translation_n("numbers",n=2))

CPU times: user 4.16 ms, sys: 180 µs, total: 4.34 ms
Wall time: 4.57 ms


5674